## Identifying and resolving conflicts in a batch of VI files

ELGs - part 10 is sets 46 to 50

There are 796 visual inspections of a total of 250 unique objects
That gives 3.184 inspections per object.

48 conflicts

In [1]:
save_filename='truth_table_ELGs_sets46_50.txt'

In [2]:
import os, sys, glob
import numpy as np

from astropy.io import fits
from astropy.table import Table, join, vstack
from astropy.io import fits

import pandas as pd
import fnmatch

import desispec.io

sys.path.append("/global/homes/r/rtojeiro/SV/python")
import VI_utils as VI

#import desispec
# library location will change ..
sys.path.append("/global/homes/r/rtojeiro/prospect/prospect/py")
from prospect import utils_specviewer,plotframes
import matplotlib.pyplot as plt 

In [3]:
tiledir   = '/global/cfs/cdirs/desi/spectro/redux/daily/tiles/'
tiles = ['67230']
nights = ['20200315']
petals = ['0','1', '2', '3', '4', '5', '6' ,'7', '8', '9']
obs_db = utils_specviewer.make_targetdict(tiledir, petals=petals, tiles=tiles) # tiles = optional 
del obs_db['67230-20200314']
obs_db.keys()

dict_keys(['67230-20200315'])

In [4]:
pd.set_option('display.max_rows', 20)

In [5]:
#set to directory with all the VI files to merge
VI_dir = os.environ['HOME']+'/SV/VI_files/SV0/ELG/'

In [6]:
vi_files = ['desi-vi_SV0_ELG_tile67230_night20200315_46_CGS.csv',
 'desi-vi_SV0_ELG_tile67230_night20200315_46_ACE.csv',
 'desi-vi_SV0_ELG_tile67230_night20200315_46_SAE.csv',
 'desi-vi_SV0_ELG_tile67230_night20200315_47_CGS.csv',
 'desi-vi_SV0_ELG_tile67230_night20200315_47_SAE.csv',
 'desi-vi_SV0_ELG_tile67230_night20200315_47_ACE.csv',
 'desi-vi_SV0_ELG_tile67230_night20200315_48_CGS.csv',
 'desi-vi_SV0_ELG_tile67230_night20200315_48_SAE.csv',
 'desi-vi_SV0_ELG_tile67230_night20200315_48_ACE.csv',
 'desi-vi_SV0_ELG_tile67230_night20200315_49_SAE.csv',
 'desi-vi_SV0_ELG_tile67230_night20200315_49_CGS.csv',
 'desi-vi_SV0_ELG_tile67230_night20200315_49_ACE.csv',
 'desi-vi_SV0_ELG_tile67230_night20200315_50_SAE.csv',
 'desi-vi_SV0_ELG_tile67230_night20200315_50_ACE.csv',
 'desi-vi_SV0_ELG_tile67230_night20200315_50_CGS.csv',
 'desi-vi_SV0_ELG_tile67230_night20200315_50_TWL.csv']

In [7]:
vi = pd.read_csv(VI_dir + vi_files[0], delimiter = " , ", engine='python')

for i in range(1,len(vi_files)):
    print(vi_files[i])
    vi2 = pd.read_csv(VI_dir + vi_files[i], delimiter = " , ", engine='python')
    vi = vi.append(vi2, ignore_index=True)
    
#make groups of visual inspections, grouped by unique objects, and state number of single and multiple VIs
vi_gp = vi.groupby(['TargetID'])
print('There are ' + str(len(vi)) + ' visual inspections of a total of ' + str(len(vi_gp)) + ' unique objects')
print('That gives ' + str(len(vi)/len(vi_gp)) + ' inspections per object.')

desi-vi_SV0_ELG_tile67230_night20200315_46_ACE.csv
desi-vi_SV0_ELG_tile67230_night20200315_46_SAE.csv
desi-vi_SV0_ELG_tile67230_night20200315_47_CGS.csv
desi-vi_SV0_ELG_tile67230_night20200315_47_SAE.csv
desi-vi_SV0_ELG_tile67230_night20200315_47_ACE.csv
desi-vi_SV0_ELG_tile67230_night20200315_48_CGS.csv
desi-vi_SV0_ELG_tile67230_night20200315_48_SAE.csv
desi-vi_SV0_ELG_tile67230_night20200315_48_ACE.csv
desi-vi_SV0_ELG_tile67230_night20200315_49_SAE.csv
desi-vi_SV0_ELG_tile67230_night20200315_49_CGS.csv
desi-vi_SV0_ELG_tile67230_night20200315_49_ACE.csv
desi-vi_SV0_ELG_tile67230_night20200315_50_SAE.csv
desi-vi_SV0_ELG_tile67230_night20200315_50_ACE.csv
desi-vi_SV0_ELG_tile67230_night20200315_50_CGS.csv
desi-vi_SV0_ELG_tile67230_night20200315_50_TWL.csv
There are 796 visual inspections of a total of 250 unique objects
That gives 3.184 inspections per object.


In [8]:
vi.keys()

Index(['TargetID', 'ExpID', 'Spec version', 'Redrock version',
       'Redrock spectype', 'Redrock z', 'VI scanner', 'VI class', 'VI issue',
       'VI z', 'VI spectype', 'VI comment'],
      dtype='object')

### Merge with zbest files
Add: fiberID, delta_chi2, flux information,.. anything else?


In [9]:

#read in fibermap info, loop over the files for all the petals
tf = Table.read(tiledir+'/'+tiles[0] + '/'+nights[0]+'/zbest-'+str(petals[0])+'-'+str(tiles[0])+'-'+nights[0]+'.fits',hdu='FIBERMAP')
tspec = Table.read(tiledir+'/'+tiles[0] + '/'+nights[0]+'/zbest-'+str(petals[0])+'-'+str(tiles[0])+'-'+nights[0]+'.fits',hdu='ZBEST')
for i in range(1,len(petals)):
    tn = Table.read(tiledir+'/'+tiles[0] + '/'+nights[0]+'/zbest-'+str(petals[i])+'-'+str(tiles[0])+'-'+nights[0]+'.fits',hdu='ZBEST')
    tnf = Table.read(tiledir+'/'+tiles[0] + '/'+nights[0]+'/zbest-'+str(petals[i])+'-'+str(tiles[0])+'-'+nights[0]+'.fits',hdu='FIBERMAP')
    tspec = vstack([tspec,tn])
    tf = vstack([tf,tnf])

tspec_df = tspec['TARGETID','DELTACHI2' ].to_pandas()
tf_df = tf['TARGETID','FIBER','FLUX_G','FLUX_R','FLUX_Z','FIBERFLUX_G','FIBERFLUX_R','FIBERFLUX_Z'].to_pandas()

tf_df = tf_df.rename(columns={"TARGETID": "TargetID"})
tspec_df = tspec_df.rename(columns={"TARGETID": "TargetID"})

vi = vi.merge(tf_df, how='left', on='TargetID')
vi = vi.merge(tspec_df, how='left', on='TargetID')

In [10]:
print(vi.keys())
print(len(vi))

Index(['TargetID', 'ExpID', 'Spec version', 'Redrock version',
       'Redrock spectype', 'Redrock z', 'VI scanner', 'VI class', 'VI issue',
       'VI z', 'VI spectype', 'VI comment', 'FIBER', 'FLUX_G', 'FLUX_R',
       'FLUX_Z', 'FIBERFLUX_G', 'FIBERFLUX_R', 'FIBERFLUX_Z', 'DELTACHI2'],
      dtype='object')
796


### Adding a bunch of useful columns

In [11]:
vi = VI.prep_new_columns(vi)

In [12]:
#check all the new columns (keys) have been added correctly
display(vi)
print(vi.keys())

,TargetID,ExpID,Spec version,Redrock version,Redrock spectype,Redrock z,VI scanner,VI class,VI issue,VI z,...,DELTACHI2,best redshift,best spectype,vi_combined_flag,vi_diff,dz,vi_combined_z,all VI comments,N_VI,merger comment
0,35185796298312209,-1,0.33.0.dev3890,0,GALAXY,0.7664,CGS,0,S,--,...,8.230352,0.7664,GALAXY,0.666667,1,0.000000,0.76640,"--|Continuum """"hump"""" at 6800-7600A. No clear ...",3,none
1,35185796302504073,-1,0.33.0.dev3890,0,QSO,4.5096,CGS,0,S,--,...,166.879404,4.5096,QSO,0.666667,1,0.000000,4.50960,"--|Continuum """"hump"""" at 6800-7600A. No clear ...",3,none
2,35185796302504279,-1,0.33.0.dev3890,0,GALAXY,0.7716,CGS,0,S,--,...,16.201880,0.7716,GALAXY,0.666667,1,0.000000,0.77160,"--| Continuum """"hump"""" at 6800-7600A. No clear...",3,none
3,35185796302504461,-1,0.33.0.dev3890,0,QSO,1.6642,CGS,0,S,--,...,7.791178,1.6642,QSO,1.000000,2,0.370967,1.66420,"--| Continuum """"hump"""" at 6800-7600A. Based o...",3,none
4,35185796302505317,-1,0.33.0.dev3890,0,GALAXY,0.6802,CGS,3,RCS,1.2569,...,61.570075,1.2569,GALAXY,2.666667,3,0.343352,0.68020,"--| Continuum """"hump"""" at 6800-7600A. Redrock...",3,none
5,35185796302505663,-1,0.33.0.dev3890,0,QSO,1.7495,CGS,1,CS,--,...,1.474611,1.7495,QSO,1.666667,2,0.270681,1.74950,"--| Continuum """"hump"""" at 6800-7600A.Based on...",3,none
6,35185796302505866,-1,0.33.0.dev3890,0,GALAXY,0.9684,CGS,4,S,--,...,702.212186,0.9684,GALAXY,3.000000,3,0.000000,0.96840,"--| Continuum """"hump"""" at 6800-7600A.|--",3,none
7,35185796302505870,-1,0.33.0.dev3890,0,GALAXY,0.5668,CGS,4,S,--,...,348.644555,0.5668,GALAXY,3.000000,3,0.000000,0.56680,"--| Continuum """"hump"""" at 6800-7600A.|--",3,none
8,35185796302506047,-1,0.33.0.dev3890,0,QSO,1.7225,CGS,4,--,--,...,422.636131,1.7225,QSO,3.333333,1,0.000000,1.72250,"QSO| Continuum """"hump"""" at 6800-7600A.|--",3,none
9,35185796302506094,-1,0.33.0.dev3890,0,GALAXY,0.8385,CGS,4,S,--,...,297.511359,0.8385,GALAXY,3.000000,3,0.000000,0.83850,"--| Continuum """"hump"""" at 6800-7600A.|--",3,none


Index(['TargetID', 'ExpID', 'Spec version', 'Redrock version',
       'Redrock spectype', 'Redrock z', 'VI scanner', 'VI class', 'VI issue',
       'VI z', 'VI spectype', 'VI comment', 'FIBER', 'FLUX_G', 'FLUX_R',
       'FLUX_Z', 'FIBERFLUX_G', 'FIBERFLUX_R', 'FIBERFLUX_Z', 'DELTACHI2',
       'best redshift', 'best spectype', 'vi_combined_flag', 'vi_diff', 'dz',
       'vi_combined_z', 'all VI comments', 'N_VI', 'merger comment'],
      dtype='object')


Get a table that holds only the objects that have been inspected more than once, and for which the individual VI classifications differ by 2 or more, or delta z / (1 + z) > 0.0033, or there is disagreement in best spectype (these are the conflicts to resolve)

In [13]:
vi_gp = vi.groupby(['TargetID'])
vi_conflict = VI.find_conflicts(vi_gp)

Get the target IDs of the problematic objects and display in table form for a quick summary:

In [14]:
unique_targets = np.unique(vi_conflict['TargetID'].tolist())
print('Targets with problematic VI: ', unique_targets)
print('Total number of conflicts to resolve: ', len(unique_targets))

Targets with problematic VI:  [35185790300457011 35185790304649454 35185790304650664 35185796302505317
 35185796302505663 35185796302505866 35185796302505870 35185796302506094
 35185796302506587 35185796302506614 35185796302507140 35185796302507304
 35185796302507357 35185796302507522 35185796302507657 35185796302508215
 35185796302508232 35185796302508514 35185796302508676 35185796302509047
 35185796306698361 35185796306699728 35185796306700539 35185796306700933
 35185796310893229 35185796310895527 35185796310895750 35185796319282771
 35185796319283738 35185796319284595 35185796319286462 35185796323476508
 35185796323479460 35185796323480105 35185796323481214 35185796327671890
 35185796327672365 35185796327672497 35185796327676001 35185796331864935
 35185796331865722 35185796331866416 35185796331867149 35185796331868052
 35185796331868872 35185796336062019 35185796336062964 35185796340253548]
Total number of conflicts to resolve:  48


In [15]:
unique_targets[1]

35185790304649454

## This is where I resolve things manually - with care!!
### I think it's better to keep it in a notebook, as typos can be backtracked rather than a single manual edit of a text file

We edit either 'VI class', or 'best redshift' to resolve conflict. At the end, we look for conflicts again and we should find none.


In [16]:
#function to display the conflict in table format and open a prospect window
def display_conflict(conflict_id, show_spectra=None):
    
    #first, remind myself of the problem:
    display(vi[vi.TargetID==unique_targets[conflict_id]][['TargetID', 
       'Redrock spectype', 'Redrock z', 'VI scanner', 'VI class', 'VI issue',
       'VI z', 'VI spectype', 'VI comment',
       'best redshift', 'best spectype', 'vi_combined_flag', 'vi_diff',
       'vi_combined_z', 'dz', 'all VI comments', 'N_VI', 'merger comment']])

    # VI interface in notebook
    if show_spectra:
        spectra, zcat= utils_specviewer.load_spectra_zcat_from_targets([unique_targets[conflict_id]], tiledir, obs_db)
        plotframes.plotspectra(spectra, zcatalog=zcat, title='Target_select', notebook=True, mask_type='CMX_TARGET',with_vi_widgets=False)

In [17]:
#first, keep a safe copy of the original dataframe
vi_safe = vi.copy()

We will inspect each conflict on a prospect window, and resolve each conflict in turn

In [18]:
#Keep track of the conflicts by conflict_id

#copy this text to a new cell to display the conflict
conflict_id=0
display_conflict(conflict_id, show_spectra=False)

,TargetID,Redrock spectype,Redrock z,VI scanner,VI class,VI issue,VI z,VI spectype,VI comment,best redshift,best spectype,vi_combined_flag,vi_diff,vi_combined_z,dz,all VI comments,N_VI,merger comment
18,35185790300457011,GALAXY,0.7396,CGS,4,RC,1.3366,--,--,1.3366,GALAXY,3.0,3,0.7396,0.343182,--| Redrock confuses OII for OII.|--,3,none
67,35185790300457011,GALAXY,0.7396,ACE,4,R,1.3366,GALAXY,Redrock confuses OII for OII.,1.3366,GALAXY,3.0,3,0.7396,0.343182,--| Redrock confuses OII for OII.|--,3,none
117,35185790300457011,GALAXY,0.7396,SAE,1,--,--,--,--,0.7396,GALAXY,3.0,3,0.7396,0.343182,--| Redrock confuses OII for OII.|--,3,none


In [19]:
#copy this text to a new cell to resolve the conflict
vi.loc[vi.TargetID==unique_targets[conflict_id], 'best redshift'] = 1.3366
vi.loc[vi.TargetID==unique_targets[conflict_id], 'VI class'] = 4
#vi.loc[vi.TargetID==unique_targets[conflict_id], 'best spectype'] = 

#add a comment
vi.loc[vi.TargetID==unique_targets[conflict_id], 'merger comment'] = 'disagreement in z. 2/3 inspectors give secure VI z based on doublet missed by OII'

#look at the values again to make sure all is well
display(vi[vi.TargetID==unique_targets[conflict_id]])
print('conflict ID = ', conflict_id)

,TargetID,ExpID,Spec version,Redrock version,Redrock spectype,Redrock z,VI scanner,VI class,VI issue,VI z,...,DELTACHI2,best redshift,best spectype,vi_combined_flag,vi_diff,dz,vi_combined_z,all VI comments,N_VI,merger comment
18,35185790300457011,-1,0.33.0.dev3890,0,GALAXY,0.7396,CGS,4,RC,1.3366,...,8.907231,1.3366,GALAXY,3.0,3,0.343182,0.7396,--| Redrock confuses OII for OII.|--,3,disagreement in z. 2/3 inspectors give secure ...
67,35185790300457011,-1,0.33.0.dev3890,0,GALAXY,0.7396,ACE,4,R,1.3366,...,8.907231,1.3366,GALAXY,3.0,3,0.343182,0.7396,--| Redrock confuses OII for OII.|--,3,disagreement in z. 2/3 inspectors give secure ...
117,35185790300457011,-1,0.33.0.dev3890,0,GALAXY,0.7396,SAE,4,--,--,...,8.907231,1.3366,GALAXY,3.0,3,0.343182,0.7396,--| Redrock confuses OII for OII.|--,3,disagreement in z. 2/3 inspectors give secure ...


conflict ID =  0


In [20]:
#copy this text to a new cell to display the conflict
conflict_id=1
display_conflict(conflict_id, show_spectra=False)

,TargetID,Redrock spectype,Redrock z,VI scanner,VI class,VI issue,VI z,VI spectype,VI comment,best redshift,best spectype,vi_combined_flag,vi_diff,vi_combined_z,dz,all VI comments,N_VI,merger comment
33,35185790304649454,GALAXY,1.6936,CGS,1,CS,--,--,--,1.6936,GALAXY,2.0,3,1.6936,0.276285,--| BAL QSO|--,3,none
82,35185790304649454,GALAXY,1.6936,ACE,4,RC,2.4378,QSO,BAL QSO,2.4378,QSO,2.0,3,1.6936,0.276285,--| BAL QSO|--,3,none
131,35185790304649454,GALAXY,1.6936,SAE,1,R,--,--,--,1.6936,GALAXY,2.0,3,1.6936,0.276285,--| BAL QSO|--,3,none


In [21]:
#copy this text to a new cell to resolve the conflict
vi.loc[vi.TargetID==unique_targets[conflict_id], 'best redshift'] = 2.4378
vi.loc[vi.TargetID==unique_targets[conflict_id], 'VI class'] = 2
vi.loc[vi.TargetID==unique_targets[conflict_id], 'best spectype'] = 'QSO'

#add a comment
vi.loc[vi.TargetID==unique_targets[conflict_id], 'merger comment'] = 'Disagreement in flag and z. 1/3 inspectors suggests BAL, but Tam is not entirely convinced. Take with flag 2.'

#look at the values again to make sure all is well
display(vi[vi.TargetID==unique_targets[conflict_id]])
print('conflict ID = ', conflict_id)


,TargetID,ExpID,Spec version,Redrock version,Redrock spectype,Redrock z,VI scanner,VI class,VI issue,VI z,...,DELTACHI2,best redshift,best spectype,vi_combined_flag,vi_diff,dz,vi_combined_z,all VI comments,N_VI,merger comment
33,35185790304649454,-1,0.33.0.dev3890,0,GALAXY,1.6936,CGS,2,CS,--,...,11.473927,2.4378,QSO,2.0,3,0.276285,1.6936,--| BAL QSO|--,3,Disagreement in flag and z. 1/3 inspectors sug...
82,35185790304649454,-1,0.33.0.dev3890,0,GALAXY,1.6936,ACE,2,RC,2.4378,...,11.473927,2.4378,QSO,2.0,3,0.276285,1.6936,--| BAL QSO|--,3,Disagreement in flag and z. 1/3 inspectors sug...
131,35185790304649454,-1,0.33.0.dev3890,0,GALAXY,1.6936,SAE,2,R,--,...,11.473927,2.4378,QSO,2.0,3,0.276285,1.6936,--| BAL QSO|--,3,Disagreement in flag and z. 1/3 inspectors sug...


conflict ID =  1


In [22]:
#copy this text to a new cell to display the conflict
conflict_id=2
display_conflict(conflict_id, show_spectra=False)


,TargetID,Redrock spectype,Redrock z,VI scanner,VI class,VI issue,VI z,VI spectype,VI comment,best redshift,best spectype,vi_combined_flag,vi_diff,vi_combined_z,dz,all VI comments,N_VI,merger comment
40,35185790304650664,GALAXY,0.8284,CGS,4,--,--,--,--,0.8284,GALAXY,4.0,0,0.8284,0.0,--| Broad MgII| MgII is missed by the model,3,none
89,35185790304650664,GALAXY,0.8284,ACE,4,C,0.8284,QSO,Broad MgII,0.8284,QSO,4.0,0,0.8284,0.0,--| Broad MgII| MgII is missed by the model,3,none
138,35185790304650664,GALAXY,0.8284,SAE,4,C,0.8284,QSO,MgII is missed by the model,0.8284,QSO,4.0,0,0.8284,0.0,--| Broad MgII| MgII is missed by the model,3,none


In [23]:
#copy this text to a new cell to resolve the conflict
#vi.loc[vi.TargetID==unique_targets[conflict_id], 'best redshift'] = 
#vi.loc[vi.TargetID==unique_targets[conflict_id], 'VI class'] = 
vi.loc[vi.TargetID==unique_targets[conflict_id], 'best spectype'] = 'QSO'

#add a comment
vi.loc[vi.TargetID==unique_targets[conflict_id], 'merger comment'] = 'disagreement in spectype. Set to QSO, see broad MgII'

#look at the values again to make sure all is well
display(vi[vi.TargetID==unique_targets[conflict_id]])
print('conflict ID = ', conflict_id)


,TargetID,ExpID,Spec version,Redrock version,Redrock spectype,Redrock z,VI scanner,VI class,VI issue,VI z,...,DELTACHI2,best redshift,best spectype,vi_combined_flag,vi_diff,dz,vi_combined_z,all VI comments,N_VI,merger comment
40,35185790304650664,-1,0.33.0.dev3890,0,GALAXY,0.8284,CGS,4,--,--,...,292.354911,0.8284,QSO,4.0,0,0.0,0.8284,--| Broad MgII| MgII is missed by the model,3,"disagreement in spectype. Set to QSO, see broa..."
89,35185790304650664,-1,0.33.0.dev3890,0,GALAXY,0.8284,ACE,4,C,0.8284,...,292.354911,0.8284,QSO,4.0,0,0.0,0.8284,--| Broad MgII| MgII is missed by the model,3,"disagreement in spectype. Set to QSO, see broa..."
138,35185790304650664,-1,0.33.0.dev3890,0,GALAXY,0.8284,SAE,4,C,0.8284,...,292.354911,0.8284,QSO,4.0,0,0.0,0.8284,--| Broad MgII| MgII is missed by the model,3,"disagreement in spectype. Set to QSO, see broa..."


conflict ID =  2


In [24]:
#copy this text to a new cell to display the conflict
conflict_id=3
display_conflict(conflict_id, show_spectra=False)

,TargetID,Redrock spectype,Redrock z,VI scanner,VI class,VI issue,VI z,VI spectype,VI comment,best redshift,best spectype,vi_combined_flag,vi_diff,vi_combined_z,dz,all VI comments,N_VI,merger comment
4,35185796302505317,GALAXY,0.6802,CGS,3,RCS,1.2569,--,--,1.2569,GALAXY,2.666667,3,0.6802,0.343352,"--| Continuum """"hump"""" at 6800-7600A. Redrock...",3,none
54,35185796302505317,GALAXY,0.6802,ACE,4,R,1.2571,GALAXY,"Continuum """"hump"""" at 6800-7600A. Redrock co...",1.2571,GALAXY,2.666667,3,0.6802,0.343352,"--| Continuum """"hump"""" at 6800-7600A. Redrock...",3,none
103,35185796302505317,GALAXY,0.6802,SAE,1,R,--,--,--,0.6802,GALAXY,2.666667,3,0.6802,0.343352,"--| Continuum """"hump"""" at 6800-7600A. Redrock...",3,none


In [25]:
#copy this text to a new cell to resolve the conflict
vi.loc[vi.TargetID==unique_targets[conflict_id], 'best redshift'] = 1.257
vi.loc[vi.TargetID==unique_targets[conflict_id], 'VI class'] = 3.5
#vi.loc[vi.TargetID==unique_targets[conflict_id], 'best spectype'] = 

#add a comment
vi.loc[vi.TargetID==unique_targets[conflict_id], 'merger comment'] = 'disagreement in z. 2/3 inspectors suggest VI z based on secure OII. poor sky subtraction.'

#look at the values again to make sure all is well
display(vi[vi.TargetID==unique_targets[conflict_id]])
print('conflict ID = ', conflict_id)

,TargetID,ExpID,Spec version,Redrock version,Redrock spectype,Redrock z,VI scanner,VI class,VI issue,VI z,...,DELTACHI2,best redshift,best spectype,vi_combined_flag,vi_diff,dz,vi_combined_z,all VI comments,N_VI,merger comment
4,35185796302505317,-1,0.33.0.dev3890,0,GALAXY,0.6802,CGS,3.5,RCS,1.2569,...,61.570075,1.257,GALAXY,2.666667,3,0.343352,0.6802,"--| Continuum """"hump"""" at 6800-7600A. Redrock...",3,disagreement in z. 2/3 inspectors suggest VI z...
54,35185796302505317,-1,0.33.0.dev3890,0,GALAXY,0.6802,ACE,3.5,R,1.2571,...,61.570075,1.257,GALAXY,2.666667,3,0.343352,0.6802,"--| Continuum """"hump"""" at 6800-7600A. Redrock...",3,disagreement in z. 2/3 inspectors suggest VI z...
103,35185796302505317,-1,0.33.0.dev3890,0,GALAXY,0.6802,SAE,3.5,R,--,...,61.570075,1.257,GALAXY,2.666667,3,0.343352,0.6802,"--| Continuum """"hump"""" at 6800-7600A. Redrock...",3,disagreement in z. 2/3 inspectors suggest VI z...


conflict ID =  3


In [26]:
#copy this text to a new cell to display the conflict
conflict_id=4
display_conflict(conflict_id, show_spectra=False)

,TargetID,Redrock spectype,Redrock z,VI scanner,VI class,VI issue,VI z,VI spectype,VI comment,best redshift,best spectype,vi_combined_flag,vi_diff,vi_combined_z,dz,all VI comments,N_VI,merger comment
5,35185796302505663,QSO,1.7495,CGS,1.0,CS,--,--,--,1.7495,QSO,1.666667,2,1.7495,0.270681,"--| Continuum """"hump"""" at 6800-7600A.Based on...",3,none
55,35185796302505663,QSO,1.7495,ACE,3.0,R,1.1638,GALAXY,"Continuum """"hump"""" at 6800-7600A.Based on cl...",1.1638,GALAXY,1.666667,2,1.7495,0.270681,"--| Continuum """"hump"""" at 6800-7600A.Based on...",3,none
104,35185796302505663,QSO,1.7495,SAE,1.0,--,--,--,--,1.7495,QSO,1.666667,2,1.7495,0.270681,"--| Continuum """"hump"""" at 6800-7600A.Based on...",3,none


In [27]:
print(vi.loc[55]['VI comment'])

  Continuum ""hump"" at 6800-7600A.Based on clear OII


In [28]:
#copy this text to a new cell to resolve the conflict
vi.loc[vi.TargetID==unique_targets[conflict_id], 'best redshift'] = 1.1725
vi.loc[vi.TargetID==unique_targets[conflict_id], 'VI class'] = 2
vi.loc[vi.TargetID==unique_targets[conflict_id], 'best spectype'] = 'GALAXY'

#add a comment
vi.loc[vi.TargetID==unique_targets[conflict_id], 'merger comment'] = 'disagreement in z. 1/3 inspectors suggest VI z based on compromised OII. I suggest a new one based on a weak OII line. Not a QSO'

#look at the values again to make sure all is well
display(vi[vi.TargetID==unique_targets[conflict_id]])
print('conflict ID = ', conflict_id)


,TargetID,ExpID,Spec version,Redrock version,Redrock spectype,Redrock z,VI scanner,VI class,VI issue,VI z,...,DELTACHI2,best redshift,best spectype,vi_combined_flag,vi_diff,dz,vi_combined_z,all VI comments,N_VI,merger comment
5,35185796302505663,-1,0.33.0.dev3890,0,QSO,1.7495,CGS,2.0,CS,--,...,1.474611,1.1725,GALAXY,1.666667,2,0.270681,1.7495,"--| Continuum """"hump"""" at 6800-7600A.Based on...",3,disagreement in z. 1/3 inspectors suggest VI z...
55,35185796302505663,-1,0.33.0.dev3890,0,QSO,1.7495,ACE,2.0,R,1.1638,...,1.474611,1.1725,GALAXY,1.666667,2,0.270681,1.7495,"--| Continuum """"hump"""" at 6800-7600A.Based on...",3,disagreement in z. 1/3 inspectors suggest VI z...
104,35185796302505663,-1,0.33.0.dev3890,0,QSO,1.7495,SAE,2.0,--,--,...,1.474611,1.1725,GALAXY,1.666667,2,0.270681,1.7495,"--| Continuum """"hump"""" at 6800-7600A.Based on...",3,disagreement in z. 1/3 inspectors suggest VI z...


conflict ID =  4


In [29]:
#copy this text to a new cell to display the conflict
conflict_id=5
display_conflict(conflict_id, show_spectra=False)

,TargetID,Redrock spectype,Redrock z,VI scanner,VI class,VI issue,VI z,VI spectype,VI comment,best redshift,best spectype,vi_combined_flag,vi_diff,vi_combined_z,dz,all VI comments,N_VI,merger comment
6,35185796302505866,GALAXY,0.9684,CGS,4.0,S,--,--,--,0.9684,GALAXY,3.0,3,0.9684,0.0,"--| Continuum """"hump"""" at 6800-7600A.|--",3,none
56,35185796302505866,GALAXY,0.9684,ACE,4.0,--,--,--,"Continuum """"hump"""" at 6800-7600A.",0.9684,GALAXY,3.0,3,0.9684,0.0,"--| Continuum """"hump"""" at 6800-7600A.|--",3,none
105,35185796302505866,GALAXY,0.9684,SAE,1.0,--,--,--,--,0.9684,GALAXY,3.0,3,0.9684,0.0,"--| Continuum """"hump"""" at 6800-7600A.|--",3,none


In [30]:
#copy this text to a new cell to resolve the conflict
#vi.loc[vi.TargetID==unique_targets[conflict_id], 'best redshift'] = 
vi.loc[vi.TargetID==unique_targets[conflict_id], 'VI class'] = 4
#vi.loc[vi.TargetID==unique_targets[conflict_id], 'best spectype'] = 

#add a comment
vi.loc[vi.TargetID==unique_targets[conflict_id], 'merger comment'] = 'disagreement in flag, but z is secure.'

#look at the values again to make sure all is well
display(vi[vi.TargetID==unique_targets[conflict_id]])
print('conflict ID = ', conflict_id)



,TargetID,ExpID,Spec version,Redrock version,Redrock spectype,Redrock z,VI scanner,VI class,VI issue,VI z,...,DELTACHI2,best redshift,best spectype,vi_combined_flag,vi_diff,dz,vi_combined_z,all VI comments,N_VI,merger comment
6,35185796302505866,-1,0.33.0.dev3890,0,GALAXY,0.9684,CGS,4.0,S,--,...,702.212186,0.9684,GALAXY,3.0,3,0.0,0.9684,"--| Continuum """"hump"""" at 6800-7600A.|--",3,"disagreement in flag, but z is secure."
56,35185796302505866,-1,0.33.0.dev3890,0,GALAXY,0.9684,ACE,4.0,--,--,...,702.212186,0.9684,GALAXY,3.0,3,0.0,0.9684,"--| Continuum """"hump"""" at 6800-7600A.|--",3,"disagreement in flag, but z is secure."
105,35185796302505866,-1,0.33.0.dev3890,0,GALAXY,0.9684,SAE,4.0,--,--,...,702.212186,0.9684,GALAXY,3.0,3,0.0,0.9684,"--| Continuum """"hump"""" at 6800-7600A.|--",3,"disagreement in flag, but z is secure."


conflict ID =  5


In [31]:
#copy this text to a new cell to display the conflict
conflict_id=6
display_conflict(conflict_id, show_spectra=False)

,TargetID,Redrock spectype,Redrock z,VI scanner,VI class,VI issue,VI z,VI spectype,VI comment,best redshift,best spectype,vi_combined_flag,vi_diff,vi_combined_z,dz,all VI comments,N_VI,merger comment
7,35185796302505870,GALAXY,0.5668,CGS,4.0,S,--,--,--,0.5668,GALAXY,3.0,3,0.5668,0.0,"--| Continuum """"hump"""" at 6800-7600A.|--",3,none
57,35185796302505870,GALAXY,0.5668,ACE,4.0,--,--,--,"Continuum """"hump"""" at 6800-7600A.",0.5668,GALAXY,3.0,3,0.5668,0.0,"--| Continuum """"hump"""" at 6800-7600A.|--",3,none
106,35185796302505870,GALAXY,0.5668,SAE,1.0,--,--,--,--,0.5668,GALAXY,3.0,3,0.5668,0.0,"--| Continuum """"hump"""" at 6800-7600A.|--",3,none


In [32]:
#copy this text to a new cell to resolve the conflict
#vi.loc[vi.TargetID==unique_targets[conflict_id], 'best redshift'] = 
vi.loc[vi.TargetID==unique_targets[conflict_id], 'VI class'] = 3
#vi.loc[vi.TargetID==unique_targets[conflict_id], 'best spectype'] = 

#add a comment
vi.loc[vi.TargetID==unique_targets[conflict_id], 'merger comment'] = 'flag disagreement, take mean. z is secure.'

#look at the values again to make sure all is well
display(vi[vi.TargetID==unique_targets[conflict_id]])
print('conflict ID = ', conflict_id)


,TargetID,ExpID,Spec version,Redrock version,Redrock spectype,Redrock z,VI scanner,VI class,VI issue,VI z,...,DELTACHI2,best redshift,best spectype,vi_combined_flag,vi_diff,dz,vi_combined_z,all VI comments,N_VI,merger comment
7,35185796302505870,-1,0.33.0.dev3890,0,GALAXY,0.5668,CGS,3.0,S,--,...,348.644555,0.5668,GALAXY,3.0,3,0.0,0.5668,"--| Continuum """"hump"""" at 6800-7600A.|--",3,"flag disagreement, take mean. z is secure."
57,35185796302505870,-1,0.33.0.dev3890,0,GALAXY,0.5668,ACE,3.0,--,--,...,348.644555,0.5668,GALAXY,3.0,3,0.0,0.5668,"--| Continuum """"hump"""" at 6800-7600A.|--",3,"flag disagreement, take mean. z is secure."
106,35185796302505870,-1,0.33.0.dev3890,0,GALAXY,0.5668,SAE,3.0,--,--,...,348.644555,0.5668,GALAXY,3.0,3,0.0,0.5668,"--| Continuum """"hump"""" at 6800-7600A.|--",3,"flag disagreement, take mean. z is secure."


conflict ID =  6


In [33]:
#copy this text to a new cell to display the conflict
conflict_id=7
display_conflict(conflict_id, show_spectra=False)


,TargetID,Redrock spectype,Redrock z,VI scanner,VI class,VI issue,VI z,VI spectype,VI comment,best redshift,best spectype,vi_combined_flag,vi_diff,vi_combined_z,dz,all VI comments,N_VI,merger comment
9,35185796302506094,GALAXY,0.8385,CGS,4.0,S,--,--,--,0.8385,GALAXY,3.0,3,0.8385,0.0,"--| Continuum """"hump"""" at 6800-7600A.|--",3,none
59,35185796302506094,GALAXY,0.8385,ACE,4.0,--,--,--,"Continuum """"hump"""" at 6800-7600A.",0.8385,GALAXY,3.0,3,0.8385,0.0,"--| Continuum """"hump"""" at 6800-7600A.|--",3,none
108,35185796302506094,GALAXY,0.8385,SAE,1.0,C,--,--,--,0.8385,GALAXY,3.0,3,0.8385,0.0,"--| Continuum """"hump"""" at 6800-7600A.|--",3,none


In [34]:
#copy this text to a new cell to resolve the conflict
#vi.loc[vi.TargetID==unique_targets[conflict_id], 'best redshift'] = 
vi.loc[vi.TargetID==unique_targets[conflict_id], 'VI class'] = 4
#vi.loc[vi.TargetID==unique_targets[conflict_id], 'best spectype'] = 

#add a comment
vi.loc[vi.TargetID==unique_targets[conflict_id], 'merger comment'] = 'flag disagreement. z is secure.'

#look at the values again to make sure all is well
display(vi[vi.TargetID==unique_targets[conflict_id]])
print('conflict ID = ', conflict_id)


,TargetID,ExpID,Spec version,Redrock version,Redrock spectype,Redrock z,VI scanner,VI class,VI issue,VI z,...,DELTACHI2,best redshift,best spectype,vi_combined_flag,vi_diff,dz,vi_combined_z,all VI comments,N_VI,merger comment
9,35185796302506094,-1,0.33.0.dev3890,0,GALAXY,0.8385,CGS,4.0,S,--,...,297.511359,0.8385,GALAXY,3.0,3,0.0,0.8385,"--| Continuum """"hump"""" at 6800-7600A.|--",3,flag disagreement. z is secure.
59,35185796302506094,-1,0.33.0.dev3890,0,GALAXY,0.8385,ACE,4.0,--,--,...,297.511359,0.8385,GALAXY,3.0,3,0.0,0.8385,"--| Continuum """"hump"""" at 6800-7600A.|--",3,flag disagreement. z is secure.
108,35185796302506094,-1,0.33.0.dev3890,0,GALAXY,0.8385,SAE,4.0,C,--,...,297.511359,0.8385,GALAXY,3.0,3,0.0,0.8385,"--| Continuum """"hump"""" at 6800-7600A.|--",3,flag disagreement. z is secure.


conflict ID =  7


In [35]:
#copy this text to a new cell to display the conflict
conflict_id=8
display_conflict(conflict_id, show_spectra=False)

,TargetID,Redrock spectype,Redrock z,VI scanner,VI class,VI issue,VI z,VI spectype,VI comment,best redshift,best spectype,vi_combined_flag,vi_diff,vi_combined_z,dz,all VI comments,N_VI,merger comment
12,35185796302506587,GALAXY,1.021,CGS,2.0,S,--,--,--,1.0210,GALAXY,3.333333,2,1.021,0.086789,--| Very odd continuum at 7600-8500A. OII clea...,3,none
61,35185796302506587,GALAXY,1.021,ACE,4.0,RS,1.1964,GALAXY,Very odd continuum at 7600-8500A. OII clear i...,1.1964,GALAXY,3.333333,2,1.021,0.086789,--| Very odd continuum at 7600-8500A. OII clea...,3,none
111,35185796302506587,GALAXY,1.021,SAE,4.0,--,1.0439,GALAXY,--,1.0439,GALAXY,3.333333,2,1.021,0.086789,--| Very odd continuum at 7600-8500A. OII clea...,3,none


In [36]:
print(vi.loc[61]['VI comment'])


 Very odd continuum at 7600-8500A. OII clear in the centre of the poor subtraction.


In [37]:
#copy this text to a new cell to resolve the conflict
vi.loc[vi.TargetID==unique_targets[conflict_id], 'best redshift'] = 1.1964
vi.loc[vi.TargetID==unique_targets[conflict_id], 'VI class'] = 3
#vi.loc[vi.TargetID==unique_targets[conflict_id], 'best spectype'] = 

#add a comment
vi.loc[vi.TargetID==unique_targets[conflict_id], 'merger comment'] = 'disagreement in z. take one of the VI z based on OII, secure.'

#look at the values again to make sure all is well
display(vi[vi.TargetID==unique_targets[conflict_id]])
print('conflict ID = ', conflict_id)

,TargetID,ExpID,Spec version,Redrock version,Redrock spectype,Redrock z,VI scanner,VI class,VI issue,VI z,...,DELTACHI2,best redshift,best spectype,vi_combined_flag,vi_diff,dz,vi_combined_z,all VI comments,N_VI,merger comment
12,35185796302506587,-1,0.33.0.dev3890,0,GALAXY,1.021,CGS,3.0,S,--,...,869.387492,1.1964,GALAXY,3.333333,2,0.086789,1.021,--| Very odd continuum at 7600-8500A. OII clea...,3,disagreement in z. take one of the VI z based ...
61,35185796302506587,-1,0.33.0.dev3890,0,GALAXY,1.021,ACE,3.0,RS,1.1964,...,869.387492,1.1964,GALAXY,3.333333,2,0.086789,1.021,--| Very odd continuum at 7600-8500A. OII clea...,3,disagreement in z. take one of the VI z based ...
111,35185796302506587,-1,0.33.0.dev3890,0,GALAXY,1.021,SAE,3.0,--,1.0439,...,869.387492,1.1964,GALAXY,3.333333,2,0.086789,1.021,--| Very odd continuum at 7600-8500A. OII clea...,3,disagreement in z. take one of the VI z based ...


conflict ID =  8


In [38]:
#copy this text to a new cell to display the conflict
conflict_id=9
display_conflict(conflict_id, show_spectra=False)

,TargetID,Redrock spectype,Redrock z,VI scanner,VI class,VI issue,VI z,VI spectype,VI comment,best redshift,best spectype,vi_combined_flag,vi_diff,vi_combined_z,dz,all VI comments,N_VI,merger comment
13,35185796302506614,GALAXY,0.5339,CGS,4.0,RS,1.0604,--,--,1.0604,GALAXY,3.0,3,0.5339,0.343243,"--| Continuum """"hump"""" at 6800-7600A. Redrock ...",3,none
62,35185796302506614,GALAXY,0.5339,ACE,4.0,R,1.0604,GALAXY,"Continuum """"hump"""" at 6800-7600A. Redrock con...",1.0604,GALAXY,3.0,3,0.5339,0.343243,"--| Continuum """"hump"""" at 6800-7600A. Redrock ...",3,none
112,35185796302506614,GALAXY,0.5339,SAE,1.0,R,--,--,--,0.5339,GALAXY,3.0,3,0.5339,0.343243,"--| Continuum """"hump"""" at 6800-7600A. Redrock ...",3,none


In [39]:
#copy this text to a new cell to resolve the conflict
vi.loc[vi.TargetID==unique_targets[conflict_id], 'best redshift'] = 1.0604
vi.loc[vi.TargetID==unique_targets[conflict_id], 'VI class'] = 4
#vi.loc[vi.TargetID==unique_targets[conflict_id], 'best spectype'] = 

#add a comment
vi.loc[vi.TargetID==unique_targets[conflict_id], 'merger comment'] = '2/3 inspectors suggest secure VI z based on OII.'

#look at the values again to make sure all is well
display(vi[vi.TargetID==unique_targets[conflict_id]])
print('conflict ID = ', conflict_id)

,TargetID,ExpID,Spec version,Redrock version,Redrock spectype,Redrock z,VI scanner,VI class,VI issue,VI z,...,DELTACHI2,best redshift,best spectype,vi_combined_flag,vi_diff,dz,vi_combined_z,all VI comments,N_VI,merger comment
13,35185796302506614,-1,0.33.0.dev3890,0,GALAXY,0.5339,CGS,4.0,RS,1.0604,...,152.73316,1.0604,GALAXY,3.0,3,0.343243,0.5339,"--| Continuum """"hump"""" at 6800-7600A. Redrock ...",3,2/3 inspectors suggest secure VI z based on OII.
62,35185796302506614,-1,0.33.0.dev3890,0,GALAXY,0.5339,ACE,4.0,R,1.0604,...,152.73316,1.0604,GALAXY,3.0,3,0.343243,0.5339,"--| Continuum """"hump"""" at 6800-7600A. Redrock ...",3,2/3 inspectors suggest secure VI z based on OII.
112,35185796302506614,-1,0.33.0.dev3890,0,GALAXY,0.5339,SAE,4.0,R,--,...,152.73316,1.0604,GALAXY,3.0,3,0.343243,0.5339,"--| Continuum """"hump"""" at 6800-7600A. Redrock ...",3,2/3 inspectors suggest secure VI z based on OII.


conflict ID =  9


In [40]:
#copy this text to a new cell to display the conflict
conflict_id=10
display_conflict(conflict_id, show_spectra=False)


,TargetID,Redrock spectype,Redrock z,VI scanner,VI class,VI issue,VI z,VI spectype,VI comment,best redshift,best spectype,vi_combined_flag,vi_diff,vi_combined_z,dz,all VI comments,N_VI,merger comment
148,35185796302507140,GALAXY,0.7779,CGS,4.0,RCS,1.2012,--,--,1.2012,GALAXY,3.666667,1,0.7779,0.238315,"--|flux bump at 6600-7600AA|Continuum """"hump""""...",3,none
198,35185796302507140,GALAXY,0.7779,SAE,3.0,R,1.2016,GALAXY,flux bump at 6600-7600AA,1.2016,GALAXY,3.666667,1,0.7779,0.238315,"--|flux bump at 6600-7600AA|Continuum """"hump""""...",3,none
248,35185796302507140,GALAXY,0.7779,ACE,4.0,R,--,--,"Continuum """"hump"""" at 6800-7600A. Redrock miss...",0.7779,GALAXY,3.666667,1,0.7779,0.238315,"--|flux bump at 6600-7600AA|Continuum """"hump""""...",3,none


In [41]:
#copy this text to a new cell to resolve the conflict
vi.loc[vi.TargetID==unique_targets[conflict_id], 'best redshift'] = 1.2014
vi.loc[vi.TargetID==unique_targets[conflict_id], 'VI class'] = 3
#vi.loc[vi.TargetID==unique_targets[conflict_id], 'best spectype'] = 

#add a comment
vi.loc[vi.TargetID==unique_targets[conflict_id], 'merger comment'] = '2/3 inspectors give VI z based on secure OII'

#look at the values again to make sure all is well
display(vi[vi.TargetID==unique_targets[conflict_id]])
print('conflict ID = ', conflict_id)

,TargetID,ExpID,Spec version,Redrock version,Redrock spectype,Redrock z,VI scanner,VI class,VI issue,VI z,...,DELTACHI2,best redshift,best spectype,vi_combined_flag,vi_diff,dz,vi_combined_z,all VI comments,N_VI,merger comment
148,35185796302507140,-1,0.33.0.dev3890,0,GALAXY,0.7779,CGS,3.0,RCS,1.2012,...,38.343952,1.2014,GALAXY,3.666667,1,0.238315,0.7779,"--|flux bump at 6600-7600AA|Continuum """"hump""""...",3,2/3 inspectors give VI z based on secure OII
198,35185796302507140,-1,0.33.0.dev3890,0,GALAXY,0.7779,SAE,3.0,R,1.2016,...,38.343952,1.2014,GALAXY,3.666667,1,0.238315,0.7779,"--|flux bump at 6600-7600AA|Continuum """"hump""""...",3,2/3 inspectors give VI z based on secure OII
248,35185796302507140,-1,0.33.0.dev3890,0,GALAXY,0.7779,ACE,3.0,R,--,...,38.343952,1.2014,GALAXY,3.666667,1,0.238315,0.7779,"--|flux bump at 6600-7600AA|Continuum """"hump""""...",3,2/3 inspectors give VI z based on secure OII


conflict ID =  10


In [42]:
#copy this text to a new cell to display the conflict
conflict_id=11
display_conflict(conflict_id, show_spectra=False)

,TargetID,Redrock spectype,Redrock z,VI scanner,VI class,VI issue,VI z,VI spectype,VI comment,best redshift,best spectype,vi_combined_flag,vi_diff,vi_combined_z,dz,all VI comments,N_VI,merger comment
149,35185796302507304,GALAXY,0.607,CGS,2.0,S,--,--,--,0.6070,GALAXY,1.333333,4,0.607,0.343311,"--|--| Continuum """"hump"""" at 6800-7600A. Redro...",3,none
199,35185796302507304,GALAXY,0.607,SAE,3.0,--,1.1587,GALAXY,--,1.1587,GALAXY,1.333333,4,0.607,0.343311,"--|--| Continuum """"hump"""" at 6800-7600A. Redro...",3,none
249,35185796302507304,GALAXY,0.607,ACE,-1.0,R,1.1584,GALAXY,"Continuum """"hump"""" at 6800-7600A. Redrock con...",1.1584,GALAXY,1.333333,4,0.607,0.343311,"--|--| Continuum """"hump"""" at 6800-7600A. Redro...",3,none


In [43]:
#copy this text to a new cell to resolve the conflict
vi.loc[vi.TargetID==unique_targets[conflict_id], 'best redshift'] = 1.1585
vi.loc[vi.TargetID==unique_targets[conflict_id], 'VI class'] = 3
#vi.loc[vi.TargetID==unique_targets[conflict_id], 'best spectype'] = 

#add a comment
vi.loc[vi.TargetID==unique_targets[conflict_id], 'merger comment'] = '2/3 inspectors give same VI based on just resolved OII'

#look at the values again to make sure all is well
display(vi[vi.TargetID==unique_targets[conflict_id]])
print('conflict ID = ', conflict_id)



,TargetID,ExpID,Spec version,Redrock version,Redrock spectype,Redrock z,VI scanner,VI class,VI issue,VI z,...,DELTACHI2,best redshift,best spectype,vi_combined_flag,vi_diff,dz,vi_combined_z,all VI comments,N_VI,merger comment
149,35185796302507304,-1,0.33.0.dev3890,0,GALAXY,0.607,CGS,3.0,S,--,...,40.352075,1.1585,GALAXY,1.333333,4,0.343311,0.607,"--|--| Continuum """"hump"""" at 6800-7600A. Redro...",3,2/3 inspectors give same VI based on just reso...
199,35185796302507304,-1,0.33.0.dev3890,0,GALAXY,0.607,SAE,3.0,--,1.1587,...,40.352075,1.1585,GALAXY,1.333333,4,0.343311,0.607,"--|--| Continuum """"hump"""" at 6800-7600A. Redro...",3,2/3 inspectors give same VI based on just reso...
249,35185796302507304,-1,0.33.0.dev3890,0,GALAXY,0.607,ACE,3.0,R,1.1584,...,40.352075,1.1585,GALAXY,1.333333,4,0.343311,0.607,"--|--| Continuum """"hump"""" at 6800-7600A. Redro...",3,2/3 inspectors give same VI based on just reso...


conflict ID =  11


In [44]:
#copy this text to a new cell to display the conflict
conflict_id=12
display_conflict(conflict_id, show_spectra=False)

,TargetID,Redrock spectype,Redrock z,VI scanner,VI class,VI issue,VI z,VI spectype,VI comment,best redshift,best spectype,vi_combined_flag,vi_diff,vi_combined_z,dz,all VI comments,N_VI,merger comment
150,35185796302507357,GALAXY,0.4959,CGS,4.0,RCS,1.0093,--,--,1.0093,GALAXY,3.0,3,0.4959,0.343272,"--|--| Continuum """"hump"""" at 6800-7600A. Redro...",3,none
200,35185796302507357,GALAXY,0.4959,SAE,1.0,--,--,--,--,0.4959,GALAXY,3.0,3,0.4959,0.343272,"--|--| Continuum """"hump"""" at 6800-7600A. Redro...",3,none
250,35185796302507357,GALAXY,0.4959,ACE,4.0,R,1.0094,GALAXY,"Continuum """"hump"""" at 6800-7600A. Redrock con...",1.0094,GALAXY,3.0,3,0.4959,0.343272,"--|--| Continuum """"hump"""" at 6800-7600A. Redro...",3,none


In [45]:
#copy this text to a new cell to resolve the conflict
vi.loc[vi.TargetID==unique_targets[conflict_id], 'best redshift'] = 1.0094
vi.loc[vi.TargetID==unique_targets[conflict_id], 'VI class'] = 3
#vi.loc[vi.TargetID==unique_targets[conflict_id], 'best spectype'] = 

#add a comment
vi.loc[vi.TargetID==unique_targets[conflict_id], 'merger comment'] = '2/3 inspectors give secure VI z based on OII'

#look at the values again to make sure all is well
display(vi[vi.TargetID==unique_targets[conflict_id]])
print('conflict ID = ', conflict_id)

,TargetID,ExpID,Spec version,Redrock version,Redrock spectype,Redrock z,VI scanner,VI class,VI issue,VI z,...,DELTACHI2,best redshift,best spectype,vi_combined_flag,vi_diff,dz,vi_combined_z,all VI comments,N_VI,merger comment
150,35185796302507357,-1,0.33.0.dev3890,0,GALAXY,0.4959,CGS,3.0,RCS,1.0093,...,84.741071,1.0094,GALAXY,3.0,3,0.343272,0.4959,"--|--| Continuum """"hump"""" at 6800-7600A. Redro...",3,2/3 inspectors give secure VI z based on OII
200,35185796302507357,-1,0.33.0.dev3890,0,GALAXY,0.4959,SAE,3.0,--,--,...,84.741071,1.0094,GALAXY,3.0,3,0.343272,0.4959,"--|--| Continuum """"hump"""" at 6800-7600A. Redro...",3,2/3 inspectors give secure VI z based on OII
250,35185796302507357,-1,0.33.0.dev3890,0,GALAXY,0.4959,ACE,3.0,R,1.0094,...,84.741071,1.0094,GALAXY,3.0,3,0.343272,0.4959,"--|--| Continuum """"hump"""" at 6800-7600A. Redro...",3,2/3 inspectors give secure VI z based on OII


conflict ID =  12


In [46]:
#copy this text to a new cell to display the conflict
conflict_id=13
display_conflict(conflict_id, show_spectra=False)

,TargetID,Redrock spectype,Redrock z,VI scanner,VI class,VI issue,VI z,VI spectype,VI comment,best redshift,best spectype,vi_combined_flag,vi_diff,vi_combined_z,dz,all VI comments,N_VI,merger comment
151,35185796302507522,GALAXY,0.7727,CGS,0.0,S,--,--,--,0.7727,GALAXY,1.0,3,0.7727,0.15011,"--|--| Continuum """"hump"""" at 6800-7600A. No si...",3,none
201,35185796302507522,GALAXY,0.7727,SAE,3.0,R,1.0388,GALAXY,--,1.0388,GALAXY,1.0,3,0.7727,0.15011,"--|--| Continuum """"hump"""" at 6800-7600A. No si...",3,none
251,35185796302507522,GALAXY,0.7727,ACE,0.0,--,--,--,"Continuum """"hump"""" at 6800-7600A. No signal",0.7727,GALAXY,1.0,3,0.7727,0.15011,"--|--| Continuum """"hump"""" at 6800-7600A. No si...",3,none


In [47]:
#copy this text to a new cell to resolve the conflict
vi.loc[vi.TargetID==unique_targets[conflict_id], 'best redshift'] = 0.7727
vi.loc[vi.TargetID==unique_targets[conflict_id], 'VI class'] = 1
#vi.loc[vi.TargetID==unique_targets[conflict_id], 'best spectype'] = 

#add a comment
vi.loc[vi.TargetID==unique_targets[conflict_id], 'merger comment'] = 'disagreement in z. very low signal, take RR answer, flag 1.'

#look at the values again to make sure all is well
display(vi[vi.TargetID==unique_targets[conflict_id]])
print('conflict ID = ', conflict_id)


,TargetID,ExpID,Spec version,Redrock version,Redrock spectype,Redrock z,VI scanner,VI class,VI issue,VI z,...,DELTACHI2,best redshift,best spectype,vi_combined_flag,vi_diff,dz,vi_combined_z,all VI comments,N_VI,merger comment
151,35185796302507522,-1,0.33.0.dev3890,0,GALAXY,0.7727,CGS,1.0,S,--,...,45.378279,0.7727,GALAXY,1.0,3,0.15011,0.7727,"--|--| Continuum """"hump"""" at 6800-7600A. No si...",3,"disagreement in z. very low signal, take RR an..."
201,35185796302507522,-1,0.33.0.dev3890,0,GALAXY,0.7727,SAE,1.0,R,1.0388,...,45.378279,0.7727,GALAXY,1.0,3,0.15011,0.7727,"--|--| Continuum """"hump"""" at 6800-7600A. No si...",3,"disagreement in z. very low signal, take RR an..."
251,35185796302507522,-1,0.33.0.dev3890,0,GALAXY,0.7727,ACE,1.0,--,--,...,45.378279,0.7727,GALAXY,1.0,3,0.15011,0.7727,"--|--| Continuum """"hump"""" at 6800-7600A. No si...",3,"disagreement in z. very low signal, take RR an..."


conflict ID =  13


In [48]:
#copy this text to a new cell to display the conflict
conflict_id=14
display_conflict(conflict_id, show_spectra=False)

,TargetID,Redrock spectype,Redrock z,VI scanner,VI class,VI issue,VI z,VI spectype,VI comment,best redshift,best spectype,vi_combined_flag,vi_diff,vi_combined_z,dz,all VI comments,N_VI,merger comment
152,35185796302507657,GALAXY,0.8877,CGS,4.0,CS,--,--,--,0.8877,GALAXY,3.333333,2,0.8877,0.198019,"--|--| Continuum """"hump"""" at 6800-7600A.",3,none
202,35185796302507657,GALAXY,0.8877,SAE,2.0,R,1.2615,GALAXY,--,1.2615,GALAXY,3.333333,2,0.8877,0.198019,"--|--| Continuum """"hump"""" at 6800-7600A.",3,none
252,35185796302507657,GALAXY,0.8877,ACE,4.0,--,--,--,"Continuum """"hump"""" at 6800-7600A.",0.8877,GALAXY,3.333333,2,0.8877,0.198019,"--|--| Continuum """"hump"""" at 6800-7600A.",3,none


In [49]:
#copy this text to a new cell to resolve the conflict
vi.loc[vi.TargetID==unique_targets[conflict_id], 'best redshift'] = 0.8877
vi.loc[vi.TargetID==unique_targets[conflict_id], 'VI class'] = 4
#vi.loc[vi.TargetID==unique_targets[conflict_id], 'best spectype'] = 

#add a comment
vi.loc[vi.TargetID==unique_targets[conflict_id], 'merger comment'] = 'disagreement in z, but RR z is secure.'

#look at the values again to make sure all is well
display(vi[vi.TargetID==unique_targets[conflict_id]])
print('conflict ID = ', conflict_id)

,TargetID,ExpID,Spec version,Redrock version,Redrock spectype,Redrock z,VI scanner,VI class,VI issue,VI z,...,DELTACHI2,best redshift,best spectype,vi_combined_flag,vi_diff,dz,vi_combined_z,all VI comments,N_VI,merger comment
152,35185796302507657,-1,0.33.0.dev3890,0,GALAXY,0.8877,CGS,4.0,CS,--,...,261.2369,0.8877,GALAXY,3.333333,2,0.198019,0.8877,"--|--| Continuum """"hump"""" at 6800-7600A.",3,"disagreement in z, but RR z is secure."
202,35185796302507657,-1,0.33.0.dev3890,0,GALAXY,0.8877,SAE,4.0,R,1.2615,...,261.2369,0.8877,GALAXY,3.333333,2,0.198019,0.8877,"--|--| Continuum """"hump"""" at 6800-7600A.",3,"disagreement in z, but RR z is secure."
252,35185796302507657,-1,0.33.0.dev3890,0,GALAXY,0.8877,ACE,4.0,--,--,...,261.2369,0.8877,GALAXY,3.333333,2,0.198019,0.8877,"--|--| Continuum """"hump"""" at 6800-7600A.",3,"disagreement in z, but RR z is secure."


conflict ID =  14


In [50]:
#copy this text to a new cell to display the conflict
conflict_id=15
display_conflict(conflict_id, show_spectra=False)

,TargetID,Redrock spectype,Redrock z,VI scanner,VI class,VI issue,VI z,VI spectype,VI comment,best redshift,best spectype,vi_combined_flag,vi_diff,vi_combined_z,dz,all VI comments,N_VI,merger comment
155,35185796302508215,GALAXY,0.8463,CGS,2.0,S,--,--,--,0.8463,GALAXY,1.666667,3,0.8463,0.343444,"--|--| Continuum """"hump"""" at 6800-7600A. Redro...",3,none
205,35185796302508215,GALAXY,0.8463,SAE,0.0,--,--,--,--,0.8463,GALAXY,1.666667,3,0.8463,0.343444,"--|--| Continuum """"hump"""" at 6800-7600A. Redro...",3,none
255,35185796302508215,GALAXY,0.8463,ACE,3.0,R,1.4804,GALAXY,"Continuum """"hump"""" at 6800-7600A. Redrock con...",1.4804,GALAXY,1.666667,3,0.8463,0.343444,"--|--| Continuum """"hump"""" at 6800-7600A. Redro...",3,none


In [51]:
#copy this text to a new cell to resolve the conflict
vi.loc[vi.TargetID==unique_targets[conflict_id], 'best redshift'] = 1.4804
vi.loc[vi.TargetID==unique_targets[conflict_id], 'VI class'] = 2
#vi.loc[vi.TargetID==unique_targets[conflict_id], 'best spectype'] = 

#add a comment
vi.loc[vi.TargetID==unique_targets[conflict_id], 'merger comment'] = 'VI z based on broadened but unresolved line. z not secure. '

#look at the values again to make sure all is well
display(vi[vi.TargetID==unique_targets[conflict_id]])
print('conflict ID = ', conflict_id)



,TargetID,ExpID,Spec version,Redrock version,Redrock spectype,Redrock z,VI scanner,VI class,VI issue,VI z,...,DELTACHI2,best redshift,best spectype,vi_combined_flag,vi_diff,dz,vi_combined_z,all VI comments,N_VI,merger comment
155,35185796302508215,-1,0.33.0.dev3890,0,GALAXY,0.8463,CGS,2.0,S,--,...,71.011357,1.4804,GALAXY,1.666667,3,0.343444,0.8463,"--|--| Continuum """"hump"""" at 6800-7600A. Redro...",3,VI z based on broadened but unresolved line. z...
205,35185796302508215,-1,0.33.0.dev3890,0,GALAXY,0.8463,SAE,2.0,--,--,...,71.011357,1.4804,GALAXY,1.666667,3,0.343444,0.8463,"--|--| Continuum """"hump"""" at 6800-7600A. Redro...",3,VI z based on broadened but unresolved line. z...
255,35185796302508215,-1,0.33.0.dev3890,0,GALAXY,0.8463,ACE,2.0,R,1.4804,...,71.011357,1.4804,GALAXY,1.666667,3,0.343444,0.8463,"--|--| Continuum """"hump"""" at 6800-7600A. Redro...",3,VI z based on broadened but unresolved line. z...


conflict ID =  15


In [52]:
#copy this text to a new cell to display the conflict
conflict_id=16
display_conflict(conflict_id, show_spectra=False)


,TargetID,Redrock spectype,Redrock z,VI scanner,VI class,VI issue,VI z,VI spectype,VI comment,best redshift,best spectype,vi_combined_flag,vi_diff,vi_combined_z,dz,all VI comments,N_VI,merger comment
157,35185796302508232,QSO,1.7672,CGS,0.0,S,--,--,--,1.7672,QSO,1.0,3,1.7672,0.088977,"--|--| Continuum """"hump"""" at 6800-7600A. OII b...",3,none
207,35185796302508232,QSO,1.7672,SAE,0.0,--,--,--,--,1.7672,QSO,1.0,3,1.7672,0.088977,"--|--| Continuum """"hump"""" at 6800-7600A. OII b...",3,none
257,35185796302508232,QSO,1.7672,ACE,3.0,R,1.5411,GALAXY,"Continuum """"hump"""" at 6800-7600A. OII believa...",1.5411,GALAXY,1.0,3,1.7672,0.088977,"--|--| Continuum """"hump"""" at 6800-7600A. OII b...",3,none


In [53]:
#copy this text to a new cell to resolve the conflict
vi.loc[vi.TargetID==unique_targets[conflict_id], 'best redshift'] = 1.5411
vi.loc[vi.TargetID==unique_targets[conflict_id], 'VI class'] = 1
vi.loc[vi.TargetID==unique_targets[conflict_id], 'best spectype'] = 'GALAXY'

#add a comment
vi.loc[vi.TargetID==unique_targets[conflict_id], 'merger comment'] = 'disagreement in all. 1/3 inspectors gives VI z based on unsecure line on skylines. take with flag 1. Not a QSO.'

#look at the values again to make sure all is well
display(vi[vi.TargetID==unique_targets[conflict_id]])
print('conflict ID = ', conflict_id)

,TargetID,ExpID,Spec version,Redrock version,Redrock spectype,Redrock z,VI scanner,VI class,VI issue,VI z,...,DELTACHI2,best redshift,best spectype,vi_combined_flag,vi_diff,dz,vi_combined_z,all VI comments,N_VI,merger comment
157,35185796302508232,-1,0.33.0.dev3890,0,QSO,1.7672,CGS,1.0,S,--,...,33.538887,1.5411,GALAXY,1.0,3,0.088977,1.7672,"--|--| Continuum """"hump"""" at 6800-7600A. OII b...",3,disagreement in all. 1/3 inspectors gives VI z...
207,35185796302508232,-1,0.33.0.dev3890,0,QSO,1.7672,SAE,1.0,--,--,...,33.538887,1.5411,GALAXY,1.0,3,0.088977,1.7672,"--|--| Continuum """"hump"""" at 6800-7600A. OII b...",3,disagreement in all. 1/3 inspectors gives VI z...
257,35185796302508232,-1,0.33.0.dev3890,0,QSO,1.7672,ACE,1.0,R,1.5411,...,33.538887,1.5411,GALAXY,1.0,3,0.088977,1.7672,"--|--| Continuum """"hump"""" at 6800-7600A. OII b...",3,disagreement in all. 1/3 inspectors gives VI z...


conflict ID =  16


In [54]:
#copy this text to a new cell to display the conflict
conflict_id=17
display_conflict(conflict_id, show_spectra=False)

,TargetID,Redrock spectype,Redrock z,VI scanner,VI class,VI issue,VI z,VI spectype,VI comment,best redshift,best spectype,vi_combined_flag,vi_diff,vi_combined_z,dz,all VI comments,N_VI,merger comment
160,35185796302508514,GALAXY,1.0059,CGS,4.0,S,--,--,--,1.0059,GALAXY,2.666667,4,1.0059,0.0,"--|--| Continuum """"hump"""" at 6800-7600A.",3,none
210,35185796302508514,GALAXY,1.0059,SAE,0.0,R,--,--,--,1.0059,GALAXY,2.666667,4,1.0059,0.0,"--|--| Continuum """"hump"""" at 6800-7600A.",3,none
260,35185796302508514,GALAXY,1.0059,ACE,4.0,--,--,--,"Continuum """"hump"""" at 6800-7600A.",1.0059,GALAXY,2.666667,4,1.0059,0.0,"--|--| Continuum """"hump"""" at 6800-7600A.",3,none


In [55]:
#copy this text to a new cell to resolve the conflict
#vi.loc[vi.TargetID==unique_targets[conflict_id], 'best redshift'] = 
vi.loc[vi.TargetID==unique_targets[conflict_id], 'VI class'] = 4
#vi.loc[vi.TargetID==unique_targets[conflict_id], 'best spectype'] = 

#add a comment
vi.loc[vi.TargetID==unique_targets[conflict_id], 'merger comment'] = 'disagreement in flag, but RR z is secure.'

#look at the values again to make sure all is well
display(vi[vi.TargetID==unique_targets[conflict_id]])
print('conflict ID = ', conflict_id)

,TargetID,ExpID,Spec version,Redrock version,Redrock spectype,Redrock z,VI scanner,VI class,VI issue,VI z,...,DELTACHI2,best redshift,best spectype,vi_combined_flag,vi_diff,dz,vi_combined_z,all VI comments,N_VI,merger comment
160,35185796302508514,-1,0.33.0.dev3890,0,GALAXY,1.0059,CGS,4.0,S,--,...,31.652803,1.0059,GALAXY,2.666667,4,0.0,1.0059,"--|--| Continuum """"hump"""" at 6800-7600A.",3,"disagreement in flag, but RR z is secure."
210,35185796302508514,-1,0.33.0.dev3890,0,GALAXY,1.0059,SAE,4.0,R,--,...,31.652803,1.0059,GALAXY,2.666667,4,0.0,1.0059,"--|--| Continuum """"hump"""" at 6800-7600A.",3,"disagreement in flag, but RR z is secure."
260,35185796302508514,-1,0.33.0.dev3890,0,GALAXY,1.0059,ACE,4.0,--,--,...,31.652803,1.0059,GALAXY,2.666667,4,0.0,1.0059,"--|--| Continuum """"hump"""" at 6800-7600A.",3,"disagreement in flag, but RR z is secure."


conflict ID =  17


In [56]:
#copy this text to a new cell to display the conflict
conflict_id=18
display_conflict(conflict_id, show_spectra=False)

,TargetID,Redrock spectype,Redrock z,VI scanner,VI class,VI issue,VI z,VI spectype,VI comment,best redshift,best spectype,vi_combined_flag,vi_diff,vi_combined_z,dz,all VI comments,N_VI,merger comment
161,35185796302508676,GALAXY,0.8527,CGS,4.0,S,--,--,--,0.8527,GALAXY,2.666667,4,0.8527,0.0,"--|--|Continuum """"hump"""" at 6800-7600A.",3,none
211,35185796302508676,GALAXY,0.8527,SAE,0.0,--,--,--,--,0.8527,GALAXY,2.666667,4,0.8527,0.0,"--|--|Continuum """"hump"""" at 6800-7600A.",3,none
261,35185796302508676,GALAXY,0.8527,ACE,4.0,--,--,--,"Continuum """"hump"""" at 6800-7600A.",0.8527,GALAXY,2.666667,4,0.8527,0.0,"--|--|Continuum """"hump"""" at 6800-7600A.",3,none


In [57]:
#copy this text to a new cell to resolve the conflict
#vi.loc[vi.TargetID==unique_targets[conflict_id], 'best redshift'] = 
vi.loc[vi.TargetID==unique_targets[conflict_id], 'VI class'] = 4
#vi.loc[vi.TargetID==unique_targets[conflict_id], 'best spectype'] = 

#add a comment
vi.loc[vi.TargetID==unique_targets[conflict_id], 'merger comment'] = 'flag disagreement, but RR z is secure.'

#look at the values again to make sure all is well
display(vi[vi.TargetID==unique_targets[conflict_id]])
print('conflict ID = ', conflict_id)


,TargetID,ExpID,Spec version,Redrock version,Redrock spectype,Redrock z,VI scanner,VI class,VI issue,VI z,...,DELTACHI2,best redshift,best spectype,vi_combined_flag,vi_diff,dz,vi_combined_z,all VI comments,N_VI,merger comment
161,35185796302508676,-1,0.33.0.dev3890,0,GALAXY,0.8527,CGS,4.0,S,--,...,713.153124,0.8527,GALAXY,2.666667,4,0.0,0.8527,"--|--|Continuum """"hump"""" at 6800-7600A.",3,"flag disagreement, but RR z is secure."
211,35185796302508676,-1,0.33.0.dev3890,0,GALAXY,0.8527,SAE,4.0,--,--,...,713.153124,0.8527,GALAXY,2.666667,4,0.0,0.8527,"--|--|Continuum """"hump"""" at 6800-7600A.",3,"flag disagreement, but RR z is secure."
261,35185796302508676,-1,0.33.0.dev3890,0,GALAXY,0.8527,ACE,4.0,--,--,...,713.153124,0.8527,GALAXY,2.666667,4,0.0,0.8527,"--|--|Continuum """"hump"""" at 6800-7600A.",3,"flag disagreement, but RR z is secure."


conflict ID =  18


In [58]:
#copy this text to a new cell to display the conflict
conflict_id=19
display_conflict(conflict_id, show_spectra=False)



,TargetID,Redrock spectype,Redrock z,VI scanner,VI class,VI issue,VI z,VI spectype,VI comment,best redshift,best spectype,vi_combined_flag,vi_diff,vi_combined_z,dz,all VI comments,N_VI,merger comment
163,35185796302509047,GALAXY,0.7513,CGS,4.0,S,--,--,--,0.7513,GALAXY,4.0,0,0.7513,0.147148,"--|--| Continuum """"hump"""" at 6800-7600A.",3,none
213,35185796302509047,GALAXY,0.7513,SAE,4.0,--,1.0090,GALAXY,--,1.0090,GALAXY,4.0,0,0.7513,0.147148,"--|--| Continuum """"hump"""" at 6800-7600A.",3,none
263,35185796302509047,GALAXY,0.7513,ACE,4.0,--,--,--,"Continuum """"hump"""" at 6800-7600A.",0.7513,GALAXY,4.0,0,0.7513,0.147148,"--|--| Continuum """"hump"""" at 6800-7600A.",3,none


In [59]:
#copy this text to a new cell to resolve the conflict
vi.loc[vi.TargetID==unique_targets[conflict_id], 'best redshift'] = 0.7513
vi.loc[vi.TargetID==unique_targets[conflict_id], 'VI class'] = 4
#vi.loc[vi.TargetID==unique_targets[conflict_id], 'best spectype'] = 

#add a comment
vi.loc[vi.TargetID==unique_targets[conflict_id], 'merger comment'] = 'disagreement in z and flag, but RR z is secure.'

#look at the values again to make sure all is well
display(vi[vi.TargetID==unique_targets[conflict_id]])
print('conflict ID = ', conflict_id)

,TargetID,ExpID,Spec version,Redrock version,Redrock spectype,Redrock z,VI scanner,VI class,VI issue,VI z,...,DELTACHI2,best redshift,best spectype,vi_combined_flag,vi_diff,dz,vi_combined_z,all VI comments,N_VI,merger comment
163,35185796302509047,-1,0.33.0.dev3890,0,GALAXY,0.7513,CGS,4.0,S,--,...,824.110228,0.7513,GALAXY,4.0,0,0.147148,0.7513,"--|--| Continuum """"hump"""" at 6800-7600A.",3,"disagreement in z and flag, but RR z is secure."
213,35185796302509047,-1,0.33.0.dev3890,0,GALAXY,0.7513,SAE,4.0,--,1.0090,...,824.110228,0.7513,GALAXY,4.0,0,0.147148,0.7513,"--|--| Continuum """"hump"""" at 6800-7600A.",3,"disagreement in z and flag, but RR z is secure."
263,35185796302509047,-1,0.33.0.dev3890,0,GALAXY,0.7513,ACE,4.0,--,--,...,824.110228,0.7513,GALAXY,4.0,0,0.147148,0.7513,"--|--| Continuum """"hump"""" at 6800-7600A.",3,"disagreement in z and flag, but RR z is secure."


conflict ID =  19


In [60]:
#copy this text to a new cell to display the conflict
conflict_id=20
display_conflict(conflict_id, show_spectra=False)

,TargetID,Redrock spectype,Redrock z,VI scanner,VI class,VI issue,VI z,VI spectype,VI comment,best redshift,best spectype,vi_combined_flag,vi_diff,vi_combined_z,dz,all VI comments,N_VI,merger comment
164,35185796306698361,GALAXY,0.5333,CGS,3.0,RS,1.0601,--,--,1.0601,GALAXY,3.333333,1,0.5333,0.064926,"--|--|Continuum """"hump"""" at 6800-7600A. Redro...",3,none
214,35185796306698361,GALAXY,0.5333,SAE,4.0,R,1.0596,GALAXY,--,1.0596,GALAXY,3.333333,1,0.5333,0.064926,"--|--|Continuum """"hump"""" at 6800-7600A. Redro...",3,none
264,35185796306698361,GALAXY,0.5333,ACE,3.0,R,0.9345,GALAXY,"Continuum """"hump"""" at 6800-7600A. Redrock con...",0.9345,GALAXY,3.333333,1,0.5333,0.064926,"--|--|Continuum """"hump"""" at 6800-7600A. Redro...",3,none


In [61]:
#copy this text to a new cell to resolve the conflict
vi.loc[vi.TargetID==unique_targets[conflict_id], 'best redshift'] = 1.0601
vi.loc[vi.TargetID==unique_targets[conflict_id], 'VI class'] = 3
#vi.loc[vi.TargetID==unique_targets[conflict_id], 'best spectype'] = 

#add a comment
vi.loc[vi.TargetID==unique_targets[conflict_id], 'merger comment'] = '2/3 inspectors give secure VI z based on OII.'

#look at the values again to make sure all is well
display(vi[vi.TargetID==unique_targets[conflict_id]])
print('conflict ID = ', conflict_id)

,TargetID,ExpID,Spec version,Redrock version,Redrock spectype,Redrock z,VI scanner,VI class,VI issue,VI z,...,DELTACHI2,best redshift,best spectype,vi_combined_flag,vi_diff,dz,vi_combined_z,all VI comments,N_VI,merger comment
164,35185796306698361,-1,0.33.0.dev3890,0,GALAXY,0.5333,CGS,3.0,RS,1.0601,...,64.901655,1.0601,GALAXY,3.333333,1,0.064926,0.5333,"--|--|Continuum """"hump"""" at 6800-7600A. Redro...",3,2/3 inspectors give secure VI z based on OII.
214,35185796306698361,-1,0.33.0.dev3890,0,GALAXY,0.5333,SAE,3.0,R,1.0596,...,64.901655,1.0601,GALAXY,3.333333,1,0.064926,0.5333,"--|--|Continuum """"hump"""" at 6800-7600A. Redro...",3,2/3 inspectors give secure VI z based on OII.
264,35185796306698361,-1,0.33.0.dev3890,0,GALAXY,0.5333,ACE,3.0,R,0.9345,...,64.901655,1.0601,GALAXY,3.333333,1,0.064926,0.5333,"--|--|Continuum """"hump"""" at 6800-7600A. Redro...",3,2/3 inspectors give secure VI z based on OII.


conflict ID =  20


In [62]:
#copy this text to a new cell to display the conflict
conflict_id=21
display_conflict(conflict_id, show_spectra=False)


,TargetID,Redrock spectype,Redrock z,VI scanner,VI class,VI issue,VI z,VI spectype,VI comment,best redshift,best spectype,vi_combined_flag,vi_diff,vi_combined_z,dz,all VI comments,N_VI,merger comment
170,35185796306699728,GALAXY,0.7822,CGS,-1.0,RS,1.0558,--,--,1.0558,GALAXY,2.333333,5,1.0558,0.000097,"--|flux bump at 6600-7600AA| Continuum """"hump""...",3,none
220,35185796306699728,GALAXY,0.7822,SAE,4.0,R,1.0559,GALAXY,flux bump at 6600-7600AA,1.0559,GALAXY,2.333333,5,1.0558,0.000097,"--|flux bump at 6600-7600AA| Continuum """"hump""...",3,none
270,35185796306699728,GALAXY,0.7822,ACE,4.0,R,1.0557,GALAXY,"Continuum """"hump"""" at 6800-7600A. Redrock mis...",1.0557,GALAXY,2.333333,5,1.0558,0.000097,"--|flux bump at 6600-7600AA| Continuum """"hump""...",3,none


In [63]:
#copy this text to a new cell to resolve the conflict
vi.loc[vi.TargetID==unique_targets[conflict_id], 'best redshift'] = 1.0558
vi.loc[vi.TargetID==unique_targets[conflict_id], 'VI class'] = 4
#vi.loc[vi.TargetID==unique_targets[conflict_id], 'best spectype'] = 

#add a comment
vi.loc[vi.TargetID==unique_targets[conflict_id], 'merger comment'] = '3/3 inspectors give same VI z, one forget to set the flag, z is secure'

#look at the values again to make sure all is well
display(vi[vi.TargetID==unique_targets[conflict_id]])
print('conflict ID = ', conflict_id)

,TargetID,ExpID,Spec version,Redrock version,Redrock spectype,Redrock z,VI scanner,VI class,VI issue,VI z,...,DELTACHI2,best redshift,best spectype,vi_combined_flag,vi_diff,dz,vi_combined_z,all VI comments,N_VI,merger comment
170,35185796306699728,-1,0.33.0.dev3890,0,GALAXY,0.7822,CGS,4.0,RS,1.0558,...,24.509839,1.0558,GALAXY,2.333333,5,0.000097,1.0558,"--|flux bump at 6600-7600AA| Continuum """"hump""...",3,"3/3 inspectors give same VI z, one forget to s..."
220,35185796306699728,-1,0.33.0.dev3890,0,GALAXY,0.7822,SAE,4.0,R,1.0559,...,24.509839,1.0558,GALAXY,2.333333,5,0.000097,1.0558,"--|flux bump at 6600-7600AA| Continuum """"hump""...",3,"3/3 inspectors give same VI z, one forget to s..."
270,35185796306699728,-1,0.33.0.dev3890,0,GALAXY,0.7822,ACE,4.0,R,1.0557,...,24.509839,1.0558,GALAXY,2.333333,5,0.000097,1.0558,"--|flux bump at 6600-7600AA| Continuum """"hump""...",3,"3/3 inspectors give same VI z, one forget to s..."


conflict ID =  21


In [64]:
#copy this text to a new cell to display the conflict
conflict_id=22
display_conflict(conflict_id, show_spectra=False)


,TargetID,Redrock spectype,Redrock z,VI scanner,VI class,VI issue,VI z,VI spectype,VI comment,best redshift,best spectype,vi_combined_flag,vi_diff,vi_combined_z,dz,all VI comments,N_VI,merger comment
175,35185796306700539,GALAXY,0.7573,CGS,3.0,RS,1.3605,--,--,1.3605,GALAXY,2.666667,3,0.7573,0.343254,"--|--| Continuum """"hump"""" at 6800-7600A. Redro...",3,none
225,35185796306700539,GALAXY,0.7573,SAE,1.0,R,--,--,--,0.7573,GALAXY,2.666667,3,0.7573,0.343254,"--|--| Continuum """"hump"""" at 6800-7600A. Redro...",3,none
275,35185796306700539,GALAXY,0.7573,ACE,4.0,R,1.3604,GALAXY,"Continuum """"hump"""" at 6800-7600A. Redrock con...",1.3604,GALAXY,2.666667,3,0.7573,0.343254,"--|--| Continuum """"hump"""" at 6800-7600A. Redro...",3,none


In [65]:
#copy this text to a new cell to resolve the conflict
vi.loc[vi.TargetID==unique_targets[conflict_id], 'best redshift'] = 0.7573
vi.loc[vi.TargetID==unique_targets[conflict_id], 'VI class'] = 2
#vi.loc[vi.TargetID==unique_targets[conflict_id], 'best spectype'] = 

#add a comment
vi.loc[vi.TargetID==unique_targets[conflict_id], 'merger comment'] = '2/3 inspectors give VI z based on broadened but unresolved line. No other features. Not secure. Calibration issues.'

#look at the values again to make sure all is well
display(vi[vi.TargetID==unique_targets[conflict_id]])
print('conflict ID = ', conflict_id)

,TargetID,ExpID,Spec version,Redrock version,Redrock spectype,Redrock z,VI scanner,VI class,VI issue,VI z,...,DELTACHI2,best redshift,best spectype,vi_combined_flag,vi_diff,dz,vi_combined_z,all VI comments,N_VI,merger comment
175,35185796306700539,-1,0.33.0.dev3890,0,GALAXY,0.7573,CGS,2.0,RS,1.3605,...,69.365451,0.7573,GALAXY,2.666667,3,0.343254,0.7573,"--|--| Continuum """"hump"""" at 6800-7600A. Redro...",3,2/3 inspectors give VI z based on broadened bu...
225,35185796306700539,-1,0.33.0.dev3890,0,GALAXY,0.7573,SAE,2.0,R,--,...,69.365451,0.7573,GALAXY,2.666667,3,0.343254,0.7573,"--|--| Continuum """"hump"""" at 6800-7600A. Redro...",3,2/3 inspectors give VI z based on broadened bu...
275,35185796306700539,-1,0.33.0.dev3890,0,GALAXY,0.7573,ACE,2.0,R,1.3604,...,69.365451,0.7573,GALAXY,2.666667,3,0.343254,0.7573,"--|--| Continuum """"hump"""" at 6800-7600A. Redro...",3,2/3 inspectors give VI z based on broadened bu...


conflict ID =  22


In [66]:
#copy this text to a new cell to display the conflict
conflict_id=23
display_conflict(conflict_id, show_spectra=False)

,TargetID,Redrock spectype,Redrock z,VI scanner,VI class,VI issue,VI z,VI spectype,VI comment,best redshift,best spectype,vi_combined_flag,vi_diff,vi_combined_z,dz,all VI comments,N_VI,merger comment
176,35185796306700933,GALAXY,0.4656,CGS,4.0,--,--,--,--,0.4656,GALAXY,4.0,0,0.4656,0.0,--| MgII is missed| Very strong lines includin...,3,none
226,35185796306700933,GALAXY,0.4656,SAE,4.0,C,0.4656,QSO,MgII is missed,0.4656,QSO,4.0,0,0.4656,0.0,--| MgII is missed| Very strong lines includin...,3,none
276,35185796306700933,GALAXY,0.4656,ACE,4.0,--,--,--,Very strong lines including MgII and HeI in e...,0.4656,GALAXY,4.0,0,0.4656,0.0,--| MgII is missed| Very strong lines includin...,3,none


In [67]:
print(vi.loc[276]['VI comment'])


 Very strong lines including MgII and HeI in emission.


In [68]:
#copy this text to a new cell to resolve the conflict
#vi.loc[vi.TargetID==unique_targets[conflict_id], 'best redshift'] = 
vi.loc[vi.TargetID==unique_targets[conflict_id], 'VI class'] = 3
vi.loc[vi.TargetID==unique_targets[conflict_id], 'best spectype'] = 'GALAXY'

#add a comment
vi.loc[vi.TargetID==unique_targets[conflict_id], 'merger comment'] = 'disagreement in spectype. possible narrow-line AGN, but not sure. keep template as GALAXY. z is secure.'

#look at the values again to make sure all is well
display(vi[vi.TargetID==unique_targets[conflict_id]])
print('conflict ID = ', conflict_id)

,TargetID,ExpID,Spec version,Redrock version,Redrock spectype,Redrock z,VI scanner,VI class,VI issue,VI z,...,DELTACHI2,best redshift,best spectype,vi_combined_flag,vi_diff,dz,vi_combined_z,all VI comments,N_VI,merger comment
176,35185796306700933,-1,0.33.0.dev3890,0,GALAXY,0.4656,CGS,3.0,--,--,...,58086.683962,0.4656,GALAXY,4.0,0,0.0,0.4656,--| MgII is missed| Very strong lines includin...,3,disagreement in spectype. possible narrow-line...
226,35185796306700933,-1,0.33.0.dev3890,0,GALAXY,0.4656,SAE,3.0,C,0.4656,...,58086.683962,0.4656,GALAXY,4.0,0,0.0,0.4656,--| MgII is missed| Very strong lines includin...,3,disagreement in spectype. possible narrow-line...
276,35185796306700933,-1,0.33.0.dev3890,0,GALAXY,0.4656,ACE,3.0,--,--,...,58086.683962,0.4656,GALAXY,4.0,0,0.0,0.4656,--| MgII is missed| Very strong lines includin...,3,disagreement in spectype. possible narrow-line...


conflict ID =  23


In [69]:
#copy this text to a new cell to display the conflict
conflict_id=24
display_conflict(conflict_id, show_spectra=False)


,TargetID,Redrock spectype,Redrock z,VI scanner,VI class,VI issue,VI z,VI spectype,VI comment,best redshift,best spectype,vi_combined_flag,vi_diff,vi_combined_z,dz,all VI comments,N_VI,merger comment
181,35185796310893229,GALAXY,0.7662,CGS,0.0,S,--,--,--,0.7662,GALAXY,2.0,3,0.7662,0.489469,--|the persistent flux bump at same location r...,3,none
231,35185796310893229,GALAXY,0.7662,SAE,3.0,R,1.6307,GALAXY,the persistent flux bump at same location repo...,1.6307,GALAXY,2.0,3,0.7662,0.489469,--|the persistent flux bump at same location r...,3,none
281,35185796310893229,GALAXY,0.7662,ACE,3.0,R,1.6304,GALAXY,"Continuum """"hump"""" at 6800-7600A. OII right a...",1.6304,GALAXY,2.0,3,0.7662,0.489469,--|the persistent flux bump at same location r...,3,none


In [70]:
#copy this text to a new cell to resolve the conflict
vi.loc[vi.TargetID==unique_targets[conflict_id], 'best redshift'] = 1.6304
vi.loc[vi.TargetID==unique_targets[conflict_id], 'VI class'] = 2
#vi.loc[vi.TargetID==unique_targets[conflict_id], 'best spectype'] = 

#add a comment
vi.loc[vi.TargetID==unique_targets[conflict_id], 'merger comment'] = '2/3 inspectors give same VI z based on dubious OII right at the edge of the spectrograph. Take with flag 2.'

#look at the values again to make sure all is well
display(vi[vi.TargetID==unique_targets[conflict_id]])
print('conflict ID = ', conflict_id)


,TargetID,ExpID,Spec version,Redrock version,Redrock spectype,Redrock z,VI scanner,VI class,VI issue,VI z,...,DELTACHI2,best redshift,best spectype,vi_combined_flag,vi_diff,dz,vi_combined_z,all VI comments,N_VI,merger comment
181,35185796310893229,-1,0.33.0.dev3890,0,GALAXY,0.7662,CGS,2.0,S,--,...,16.637762,1.6304,GALAXY,2.0,3,0.489469,0.7662,--|the persistent flux bump at same location r...,3,2/3 inspectors give same VI z based on dubious...
231,35185796310893229,-1,0.33.0.dev3890,0,GALAXY,0.7662,SAE,2.0,R,1.6307,...,16.637762,1.6304,GALAXY,2.0,3,0.489469,0.7662,--|the persistent flux bump at same location r...,3,2/3 inspectors give same VI z based on dubious...
281,35185796310893229,-1,0.33.0.dev3890,0,GALAXY,0.7662,ACE,2.0,R,1.6304,...,16.637762,1.6304,GALAXY,2.0,3,0.489469,0.7662,--|the persistent flux bump at same location r...,3,2/3 inspectors give same VI z based on dubious...


conflict ID =  24


In [71]:
#copy this text to a new cell to display the conflict
conflict_id=25
display_conflict(conflict_id, show_spectra=False)



,TargetID,Redrock spectype,Redrock z,VI scanner,VI class,VI issue,VI z,VI spectype,VI comment,best redshift,best spectype,vi_combined_flag,vi_diff,vi_combined_z,dz,all VI comments,N_VI,merger comment
189,35185796310895527,GALAXY,0.762,CGS,2.0,S,--,--,--,0.7620,GALAXY,3.0,2,0.762,0.343303,"--|--|Continuum """"hump"""" at 6800-7600A. Redro...",3,none
239,35185796310895527,GALAXY,0.762,SAE,4.0,R,1.3669,GALAXY,--,1.3669,GALAXY,3.0,2,0.762,0.343303,"--|--|Continuum """"hump"""" at 6800-7600A. Redro...",3,none
289,35185796310895527,GALAXY,0.762,ACE,3.0,R,1.3666,GALAXY,"Continuum """"hump"""" at 6800-7600A. Redrock con...",1.3666,GALAXY,3.0,2,0.762,0.343303,"--|--|Continuum """"hump"""" at 6800-7600A. Redro...",3,none


In [72]:
#copy this text to a new cell to resolve the conflict
vi.loc[vi.TargetID==unique_targets[conflict_id], 'best redshift'] = 1.3666
vi.loc[vi.TargetID==unique_targets[conflict_id], 'VI class'] = 2.5
#vi.loc[vi.TargetID==unique_targets[conflict_id], 'best spectype'] = 

#add a comment
vi.loc[vi.TargetID==unique_targets[conflict_id], 'merger comment'] = '2/3 inspectors give same VI z based on just resolved OII in skyline region. Take with flag 2.5 '

#look at the values again to make sure all is well
display(vi[vi.TargetID==unique_targets[conflict_id]])
print('conflict ID = ', conflict_id)


,TargetID,ExpID,Spec version,Redrock version,Redrock spectype,Redrock z,VI scanner,VI class,VI issue,VI z,...,DELTACHI2,best redshift,best spectype,vi_combined_flag,vi_diff,dz,vi_combined_z,all VI comments,N_VI,merger comment
189,35185796310895527,-1,0.33.0.dev3890,0,GALAXY,0.762,CGS,2.5,S,--,...,275.89446,1.3666,GALAXY,3.0,2,0.343303,0.762,"--|--|Continuum """"hump"""" at 6800-7600A. Redro...",3,2/3 inspectors give same VI z based on just re...
239,35185796310895527,-1,0.33.0.dev3890,0,GALAXY,0.762,SAE,2.5,R,1.3669,...,275.89446,1.3666,GALAXY,3.0,2,0.343303,0.762,"--|--|Continuum """"hump"""" at 6800-7600A. Redro...",3,2/3 inspectors give same VI z based on just re...
289,35185796310895527,-1,0.33.0.dev3890,0,GALAXY,0.762,ACE,2.5,R,1.3666,...,275.89446,1.3666,GALAXY,3.0,2,0.343303,0.762,"--|--|Continuum """"hump"""" at 6800-7600A. Redro...",3,2/3 inspectors give same VI z based on just re...


conflict ID =  25


In [73]:
#copy this text to a new cell to display the conflict
conflict_id=26
display_conflict(conflict_id, show_spectra=False)


,TargetID,Redrock spectype,Redrock z,VI scanner,VI class,VI issue,VI z,VI spectype,VI comment,best redshift,best spectype,vi_combined_flag,vi_diff,vi_combined_z,dz,all VI comments,N_VI,merger comment
190,35185796310895750,GALAXY,0.4894,CGS,2.0,S,--,--,--,0.4894,GALAXY,3.0,2,0.4894,0.310745,"--|--| Continuum """"hump"""" at 6800-7600A. Low r...",3,none
240,35185796310895750,GALAXY,0.4894,SAE,3.0,R,0.1363,GALAXY,--,0.1363,GALAXY,3.0,2,0.4894,0.310745,"--|--| Continuum """"hump"""" at 6800-7600A. Low r...",3,none
290,35185796310895750,GALAXY,0.4894,ACE,4.0,R,0.1363,GALAXY,"Continuum """"hump"""" at 6800-7600A. Low redshif...",0.1363,GALAXY,3.0,2,0.4894,0.310745,"--|--| Continuum """"hump"""" at 6800-7600A. Low r...",3,none


In [74]:
#copy this text to a new cell to resolve the conflict
vi.loc[vi.TargetID==unique_targets[conflict_id], 'best redshift'] = 0.1363
vi.loc[vi.TargetID==unique_targets[conflict_id], 'VI class'] = 3.5
#vi.loc[vi.TargetID==unique_targets[conflict_id], 'best spectype'] = 

#add a comment
vi.loc[vi.TargetID==unique_targets[conflict_id], 'merger comment'] = '2/3 inspectors give same VI z. secure.'

#look at the values again to make sure all is well
display(vi[vi.TargetID==unique_targets[conflict_id]])
print('conflict ID = ', conflict_id)


,TargetID,ExpID,Spec version,Redrock version,Redrock spectype,Redrock z,VI scanner,VI class,VI issue,VI z,...,DELTACHI2,best redshift,best spectype,vi_combined_flag,vi_diff,dz,vi_combined_z,all VI comments,N_VI,merger comment
190,35185796310895750,-1,0.33.0.dev3890,0,GALAXY,0.4894,CGS,3.5,S,--,...,110.202732,0.1363,GALAXY,3.0,2,0.310745,0.4894,"--|--| Continuum """"hump"""" at 6800-7600A. Low r...",3,2/3 inspectors give same VI z. secure.
240,35185796310895750,-1,0.33.0.dev3890,0,GALAXY,0.4894,SAE,3.5,R,0.1363,...,110.202732,0.1363,GALAXY,3.0,2,0.310745,0.4894,"--|--| Continuum """"hump"""" at 6800-7600A. Low r...",3,2/3 inspectors give same VI z. secure.
290,35185796310895750,-1,0.33.0.dev3890,0,GALAXY,0.4894,ACE,3.5,R,0.1363,...,110.202732,0.1363,GALAXY,3.0,2,0.310745,0.4894,"--|--| Continuum """"hump"""" at 6800-7600A. Low r...",3,2/3 inspectors give same VI z. secure.


conflict ID =  26


In [75]:
#copy this text to a new cell to display the conflict
conflict_id=27
display_conflict(conflict_id, show_spectra=False)



,TargetID,Redrock spectype,Redrock z,VI scanner,VI class,VI issue,VI z,VI spectype,VI comment,best redshift,best spectype,vi_combined_flag,vi_diff,vi_combined_z,dz,all VI comments,N_VI,merger comment
308,35185796319282771,GALAXY,1.5469,CGS,1.0,--,--,--,--,1.5469,GALAXY,2.666667,3,1.5469,0.0,--|--|--,3,none
358,35185796319282771,GALAXY,1.5469,SAE,3.0,--,1.5469,GALAXY,--,1.5469,GALAXY,2.666667,3,1.5469,0.0,--|--|--,3,none
408,35185796319282771,GALAXY,1.5469,ACE,4.0,--,--,--,--,1.5469,GALAXY,2.666667,3,1.5469,0.0,--|--|--,3,none


In [76]:
#copy this text to a new cell to resolve the conflict
#vi.loc[vi.TargetID==unique_targets[conflict_id], 'best redshift'] = 
vi.loc[vi.TargetID==unique_targets[conflict_id], 'VI class'] = 2.666667
#vi.loc[vi.TargetID==unique_targets[conflict_id], 'best spectype'] = 

#add a comment
vi.loc[vi.TargetID==unique_targets[conflict_id], 'merger comment'] = 'disagreement in flag. just about resolved OII. Take mean.'

#look at the values again to make sure all is well
display(vi[vi.TargetID==unique_targets[conflict_id]])
print('conflict ID = ', conflict_id)


,TargetID,ExpID,Spec version,Redrock version,Redrock spectype,Redrock z,VI scanner,VI class,VI issue,VI z,...,DELTACHI2,best redshift,best spectype,vi_combined_flag,vi_diff,dz,vi_combined_z,all VI comments,N_VI,merger comment
308,35185796319282771,-1,0.33.0.dev3890,0,GALAXY,1.5469,CGS,2.666667,--,--,...,166.476017,1.5469,GALAXY,2.666667,3,0.0,1.5469,--|--|--,3,disagreement in flag. just about resolved OII....
358,35185796319282771,-1,0.33.0.dev3890,0,GALAXY,1.5469,SAE,2.666667,--,1.5469,...,166.476017,1.5469,GALAXY,2.666667,3,0.0,1.5469,--|--|--,3,disagreement in flag. just about resolved OII....
408,35185796319282771,-1,0.33.0.dev3890,0,GALAXY,1.5469,ACE,2.666667,--,--,...,166.476017,1.5469,GALAXY,2.666667,3,0.0,1.5469,--|--|--,3,disagreement in flag. just about resolved OII....


conflict ID =  27


In [77]:
#copy this text to a new cell to display the conflict
conflict_id=28
display_conflict(conflict_id, show_spectra=False)


,TargetID,Redrock spectype,Redrock z,VI scanner,VI class,VI issue,VI z,VI spectype,VI comment,best redshift,best spectype,vi_combined_flag,vi_diff,vi_combined_z,dz,all VI comments,N_VI,merger comment
315,35185796319283738,GALAXY,1.4873,CGS,3.0,--,--,--,--,1.4873,GALAXY,2.333333,2,1.4873,0.0,--|--|--,3,none
365,35185796319283738,GALAXY,1.4873,SAE,1.0,--,--,--,--,1.4873,GALAXY,2.333333,2,1.4873,0.0,--|--|--,3,none
415,35185796319283738,GALAXY,1.4873,ACE,3.0,--,--,--,--,1.4873,GALAXY,2.333333,2,1.4873,0.0,--|--|--,3,none


In [78]:
#copy this text to a new cell to resolve the conflict
#vi.loc[vi.TargetID==unique_targets[conflict_id], 'best redshift'] = 
vi.loc[vi.TargetID==unique_targets[conflict_id], 'VI class'] = 3
#vi.loc[vi.TargetID==unique_targets[conflict_id], 'best spectype'] = 

#add a comment
vi.loc[vi.TargetID==unique_targets[conflict_id], 'merger comment'] = 'disagreement in flag, but z is secure.'

#look at the values again to make sure all is well
display(vi[vi.TargetID==unique_targets[conflict_id]])
print('conflict ID = ', conflict_id)

,TargetID,ExpID,Spec version,Redrock version,Redrock spectype,Redrock z,VI scanner,VI class,VI issue,VI z,...,DELTACHI2,best redshift,best spectype,vi_combined_flag,vi_diff,dz,vi_combined_z,all VI comments,N_VI,merger comment
315,35185796319283738,-1,0.33.0.dev3890,0,GALAXY,1.4873,CGS,3.0,--,--,...,39.069132,1.4873,GALAXY,2.333333,2,0.0,1.4873,--|--|--,3,"disagreement in flag, but z is secure."
365,35185796319283738,-1,0.33.0.dev3890,0,GALAXY,1.4873,SAE,3.0,--,--,...,39.069132,1.4873,GALAXY,2.333333,2,0.0,1.4873,--|--|--,3,"disagreement in flag, but z is secure."
415,35185796319283738,-1,0.33.0.dev3890,0,GALAXY,1.4873,ACE,3.0,--,--,...,39.069132,1.4873,GALAXY,2.333333,2,0.0,1.4873,--|--|--,3,"disagreement in flag, but z is secure."


conflict ID =  28


In [79]:
#copy this text to a new cell to display the conflict
conflict_id=29
display_conflict(conflict_id, show_spectra=False)



,TargetID,Redrock spectype,Redrock z,VI scanner,VI class,VI issue,VI z,VI spectype,VI comment,best redshift,best spectype,vi_combined_flag,vi_diff,vi_combined_z,dz,all VI comments,N_VI,merger comment
319,35185796319284595,GALAXY,0.1203,CGS,2.0,S,--,--,--,0.1203,GALAXY,2.333333,1,0.1203,0.054182,--|the persistent flux bump at same location r...,3,none
369,35185796319284595,GALAXY,0.1203,SAE,2.0,R,0.181,GALAXY,the persistent flux bump at same location repo...,0.1810,GALAXY,2.333333,1,0.1203,0.054182,--|the persistent flux bump at same location r...,3,none
419,35185796319284595,GALAXY,0.1203,ACE,3.0,--,0.1810,GALAXY,Based mainly on OII at 4200A,0.1810,GALAXY,2.333333,1,0.1203,0.054182,--|the persistent flux bump at same location r...,3,none


In [80]:
#copy this text to a new cell to resolve the conflict
vi.loc[vi.TargetID==unique_targets[conflict_id], 'best redshift'] = 0.1810
vi.loc[vi.TargetID==unique_targets[conflict_id], 'VI class'] = 2
#vi.loc[vi.TargetID==unique_targets[conflict_id], 'best spectype'] = 

#add a comment
vi.loc[vi.TargetID==unique_targets[conflict_id], 'merger comment'] = 'disagreement in z. 2/3 inspectors give VI z based on very weak features. Take with flag 2.'

#look at the values again to make sure all is well
display(vi[vi.TargetID==unique_targets[conflict_id]])
print('conflict ID = ', conflict_id)


,TargetID,ExpID,Spec version,Redrock version,Redrock spectype,Redrock z,VI scanner,VI class,VI issue,VI z,...,DELTACHI2,best redshift,best spectype,vi_combined_flag,vi_diff,dz,vi_combined_z,all VI comments,N_VI,merger comment
319,35185796319284595,-1,0.33.0.dev3890,0,GALAXY,0.1203,CGS,2.0,S,--,...,53.609108,0.181,GALAXY,2.333333,1,0.054182,0.1203,--|the persistent flux bump at same location r...,3,disagreement in z. 2/3 inspectors give VI z ba...
369,35185796319284595,-1,0.33.0.dev3890,0,GALAXY,0.1203,SAE,2.0,R,0.181,...,53.609108,0.181,GALAXY,2.333333,1,0.054182,0.1203,--|the persistent flux bump at same location r...,3,disagreement in z. 2/3 inspectors give VI z ba...
419,35185796319284595,-1,0.33.0.dev3890,0,GALAXY,0.1203,ACE,2.0,--,0.1810,...,53.609108,0.181,GALAXY,2.333333,1,0.054182,0.1203,--|the persistent flux bump at same location r...,3,disagreement in z. 2/3 inspectors give VI z ba...


conflict ID =  29


In [81]:
#copy this text to a new cell to display the conflict
conflict_id=30
display_conflict(conflict_id, show_spectra=False)

,TargetID,Redrock spectype,Redrock z,VI scanner,VI class,VI issue,VI z,VI spectype,VI comment,best redshift,best spectype,vi_combined_flag,vi_diff,vi_combined_z,dz,all VI comments,N_VI,merger comment
330,35185796319286462,GALAXY,0.2878,CGS,1.0,--,--,--,--,0.2878,GALAXY,2.333333,2,0.2878,2.067246,--|only based on strong Lyman alpha|Bright nar...,3,none
380,35185796319286462,GALAXY,0.2878,SAE,3.0,RC,2.950,QSO,only based on strong Lyman alpha,2.9500,QSO,2.333333,2,0.2878,2.067246,--|only based on strong Lyman alpha|Bright nar...,3,none
430,35185796319286462,GALAXY,0.2878,ACE,3.0,RC,2.9444,QSO,Bright narrow Lyman alpha. Not OII as Redrock ...,2.9444,QSO,2.333333,2,0.2878,2.067246,--|only based on strong Lyman alpha|Bright nar...,3,none


In [ ]:
#copy this text to a new cell to resolve the conflict
vi.loc[vi.TargetID==unique_targets[conflict_id], 'best redshift'] = 0.2878
vi.loc[vi.TargetID==unique_targets[conflict_id], 'VI class'] = 1
vi.loc[vi.TargetID==unique_targets[conflict_id], 'best spectype'] = 'GALAXY'

#add a comment
vi.loc[vi.TargetID==unique_targets[conflict_id], 'merger comment'] = 'disagreement in all. 2/3 inspectors give VI z based on potential LyA, but i am not convinced - profile does not look rigth, no other features. take RR answer, flag 1.'

#look at the values again to make sure all is well
display(vi[vi.TargetID==unique_targets[conflict_id]])
print('conflict ID = ', conflict_id)

In [ ]:
#copy this text to a new cell to display the conflict
conflict_id=31
display_conflict(conflict_id, show_spectra=False)

In [ ]:
#copy this text to a new cell to resolve the conflict
#vi.loc[vi.TargetID==unique_targets[conflict_id], 'best redshift'] = 
vi.loc[vi.TargetID==unique_targets[conflict_id], 'VI class'] = 4
#vi.loc[vi.TargetID==unique_targets[conflict_id], 'best spectype'] = 

#add a comment
vi.loc[vi.TargetID==unique_targets[conflict_id], 'merger comment'] = 'one inspector forgot to set the flag, but RR z is secure.'

#look at the values again to make sure all is well
display(vi[vi.TargetID==unique_targets[conflict_id]])
print('conflict ID = ', conflict_id)



In [ ]:
#copy this text to a new cell to display the conflict
conflict_id=32
display_conflict(conflict_id, show_spectra=False)



In [ ]:
#copy this text to a new cell to resolve the conflict
vi.loc[vi.TargetID==unique_targets[conflict_id], 'best redshift'] = 1.0400
vi.loc[vi.TargetID==unique_targets[conflict_id], 'VI class'] = 2
#vi.loc[vi.TargetID==unique_targets[conflict_id], 'best spectype'] = 

#add a comment
vi.loc[vi.TargetID==unique_targets[conflict_id], 'merger comment'] = '2/3 inspectors suggest same VI z based on single non-resolved line. Flag 2.'

#look at the values again to make sure all is well
display(vi[vi.TargetID==unique_targets[conflict_id]])
print('conflict ID = ', conflict_id)


In [ ]:
#copy this text to a new cell to display the conflict
conflict_id=33
display_conflict(conflict_id, show_spectra=False)

In [ ]:
#copy this text to a new cell to resolve the conflict
vi.loc[vi.TargetID==unique_targets[conflict_id], 'best redshift'] = 2.319
vi.loc[vi.TargetID==unique_targets[conflict_id], 'VI class'] = 3
#vi.loc[vi.TargetID==unique_targets[conflict_id], 'best spectype'] = 

#add a comment
vi.loc[vi.TargetID==unique_targets[conflict_id], 'merger comment'] = 'no data in r spectrograph. small change in z to align features more accurately proposed by 2/3 inspectors. Secure.'

#look at the values again to make sure all is well
display(vi[vi.TargetID==unique_targets[conflict_id]])
print('conflict ID = ', conflict_id)


In [ ]:
#copy this text to a new cell to display the conflict
conflict_id=34
display_conflict(conflict_id, show_spectra=False)



In [ ]:
#copy this text to a new cell to resolve the conflict
vi.loc[vi.TargetID==unique_targets[conflict_id], 'best redshift'] = 1.0392
vi.loc[vi.TargetID==unique_targets[conflict_id], 'VI class'] = 2.5
#vi.loc[vi.TargetID==unique_targets[conflict_id], 'best spectype'] = 

#add a comment
vi.loc[vi.TargetID==unique_targets[conflict_id], 'merger comment'] = '1/3 inspectors suggests VI z based on resolved line on top to noise. No other features, but the continuum is there, so i am not completely sure. set flag 2.5'

#look at the values again to make sure all is well
display(vi[vi.TargetID==unique_targets[conflict_id]])
print('conflict ID = ', conflict_id)


In [ ]:
#copy this text to a new cell to display the conflict
conflict_id=35
display_conflict(conflict_id, show_spectra=False)

In [ ]:
#copy this text to a new cell to resolve the conflict
vi.loc[vi.TargetID==unique_targets[conflict_id], 'best redshift'] = 1.2833
vi.loc[vi.TargetID==unique_targets[conflict_id], 'VI class'] = 3
#vi.loc[vi.TargetID==unique_targets[conflict_id], 'best spectype'] = 

#add a comment
vi.loc[vi.TargetID==unique_targets[conflict_id], 'merger comment'] = 'disagreement in redhift. possibly two targets in the fiber, one at the RR z and one at 0.9860. Take RR z as that is the strongest.'

#look at the values again to make sure all is well
display(vi[vi.TargetID==unique_targets[conflict_id]])
print('conflict ID = ', conflict_id)

In [ ]:
#copy this text to a new cell to display the conflict
conflict_id=36
display_conflict(conflict_id, show_spectra=False)


In [ ]:
#copy this text to a new cell to resolve the conflict
#vi.loc[vi.TargetID==unique_targets[conflict_id], 'best redshift'] = 
vi.loc[vi.TargetID==unique_targets[conflict_id], 'VI class'] = 2
#vi.loc[vi.TargetID==unique_targets[conflict_id], 'best spectype'] = 

#add a comment
vi.loc[vi.TargetID==unique_targets[conflict_id], 'merger comment'] = 'disagreement in flag. broadened but unresolved line. no other features. set to mean flag 2.'

#look at the values again to make sure all is well
display(vi[vi.TargetID==unique_targets[conflict_id]])
print('conflict ID = ', conflict_id)

In [ ]:
#copy this text to a new cell to display the conflict
conflict_id=37
display_conflict(conflict_id, show_spectra=False)


In [ ]:
#copy this text to a new cell to resolve the conflict
#vi.loc[vi.TargetID==unique_targets[conflict_id], 'best redshift'] = 
vi.loc[vi.TargetID==unique_targets[conflict_id], 'VI class'] = 3
#vi.loc[vi.TargetID==unique_targets[conflict_id], 'best spectype'] = 

#add a comment
vi.loc[vi.TargetID==unique_targets[conflict_id], 'merger comment'] = 'disagreement in flag. OII in skylines but secure.'

#look at the values again to make sure all is well
display(vi[vi.TargetID==unique_targets[conflict_id]])
print('conflict ID = ', conflict_id)



In [ ]:
#copy this text to a new cell to display the conflict
conflict_id=38
display_conflict(conflict_id, show_spectra=False)


In [98]:
#copy this text to a new cell to resolve the conflict
#vi.loc[vi.TargetID==unique_targets[conflict_id], 'best redshift'] = 
vi.loc[vi.TargetID==unique_targets[conflict_id], 'VI class'] = 2
#vi.loc[vi.TargetID==unique_targets[conflict_id], 'best spectype'] = 

#add a comment
vi.loc[vi.TargetID==unique_targets[conflict_id], 'merger comment'] = 'disagreement in flag. OII not secure, set to mean flag 2'

#look at the values again to make sure all is well
display(vi[vi.TargetID==unique_targets[conflict_id]])
print('conflict ID = ', conflict_id)


,TargetID,ExpID,Spec version,Redrock version,Redrock spectype,Redrock z,VI scanner,VI class,VI issue,VI z,...,DELTACHI2,best redshift,best spectype,vi_combined_flag,vi_diff,dz,vi_combined_z,all VI comments,N_VI,merger comment
463,35185796327676001,-1,0.33.0.dev3890,0,GALAXY,1.3843,SAE,2.0,--,--,...,11.770153,1.3843,GALAXY,2.0,2,0.0,1.3843,--|--|--,3,"disagreement in flag. OII not secure, set to m..."
512,35185796327676001,-1,0.33.0.dev3890,0,GALAXY,1.3843,CGS,2.0,--,--,...,11.770153,1.3843,GALAXY,2.0,2,0.0,1.3843,--|--|--,3,"disagreement in flag. OII not secure, set to m..."
562,35185796327676001,-1,0.33.0.dev3890,0,GALAXY,1.3843,ACE,2.0,--,--,...,11.770153,1.3843,GALAXY,2.0,2,0.0,1.3843,--|--|--,3,"disagreement in flag. OII not secure, set to m..."


conflict ID =  38


In [99]:
#copy this text to a new cell to display the conflict
conflict_id=39
display_conflict(conflict_id, show_spectra=False)


,TargetID,Redrock spectype,Redrock z,VI scanner,VI class,VI issue,VI z,VI spectype,VI comment,best redshift,best spectype,vi_combined_flag,vi_diff,vi_combined_z,dz,all VI comments,N_VI,merger comment
486,35185796331864935,GALAXY,1.5839,SAE,4.0,RC,0.0000,STAR,--,0.0000,STAR,2.333333,4,1.5839,1.5839,--|--| Possible white dwarf?,3,none
536,35185796331864935,GALAXY,1.5839,CGS,0.0,S,--,--,--,1.5839,GALAXY,2.333333,4,1.5839,1.5839,--|--| Possible white dwarf?,3,none
586,35185796331864935,GALAXY,1.5839,ACE,3.0,RC,0.0000,STAR,Possible white dwarf?,0.0000,STAR,2.333333,4,1.5839,1.5839,--|--| Possible white dwarf?,3,none


In [100]:
#copy this text to a new cell to resolve the conflict
vi.loc[vi.TargetID==unique_targets[conflict_id], 'best redshift'] = 0.0
vi.loc[vi.TargetID==unique_targets[conflict_id], 'VI class'] = 2
vi.loc[vi.TargetID==unique_targets[conflict_id], 'best spectype'] = 'STAR'

#add a comment
vi.loc[vi.TargetID==unique_targets[conflict_id], 'merger comment'] = '2/3 inspectors suggest star, possibly WD, maybe some balmer lines but very noisy. flag 2.'

#look at the values again to make sure all is well
display(vi[vi.TargetID==unique_targets[conflict_id]])
print('conflict ID = ', conflict_id)


,TargetID,ExpID,Spec version,Redrock version,Redrock spectype,Redrock z,VI scanner,VI class,VI issue,VI z,...,DELTACHI2,best redshift,best spectype,vi_combined_flag,vi_diff,dz,vi_combined_z,all VI comments,N_VI,merger comment
486,35185796331864935,-1,0.33.0.dev3890,0,GALAXY,1.5839,SAE,2.0,RC,0.0000,...,17.967034,0.0,STAR,2.333333,4,1.5839,1.5839,--|--| Possible white dwarf?,3,"2/3 inspectors suggest star, possibly WD, mayb..."
536,35185796331864935,-1,0.33.0.dev3890,0,GALAXY,1.5839,CGS,2.0,S,--,...,17.967034,0.0,STAR,2.333333,4,1.5839,1.5839,--|--| Possible white dwarf?,3,"2/3 inspectors suggest star, possibly WD, mayb..."
586,35185796331864935,-1,0.33.0.dev3890,0,GALAXY,1.5839,ACE,2.0,RC,0.0000,...,17.967034,0.0,STAR,2.333333,4,1.5839,1.5839,--|--| Possible white dwarf?,3,"2/3 inspectors suggest star, possibly WD, mayb..."


conflict ID =  39


In [101]:
#copy this text to a new cell to display the conflict
conflict_id=40
display_conflict(conflict_id, show_spectra=False)


,TargetID,Redrock spectype,Redrock z,VI scanner,VI class,VI issue,VI z,VI spectype,VI comment,best redshift,best spectype,vi_combined_flag,vi_diff,vi_combined_z,dz,all VI comments,N_VI,merger comment
490,35185796331865722,GALAXY,1.4936,SAE,1.0,--,--,--,--,1.4936,GALAXY,1.666667,2,1.4936,0.0,--|--|--,3,none
540,35185796331865722,GALAXY,1.4936,CGS,1.0,--,--,--,--,1.4936,GALAXY,1.666667,2,1.4936,0.0,--|--|--,3,none
590,35185796331865722,GALAXY,1.4936,ACE,3.0,--,--,--,--,1.4936,GALAXY,1.666667,2,1.4936,0.0,--|--|--,3,none


In [102]:
#copy this text to a new cell to resolve the conflict
#vi.loc[vi.TargetID==unique_targets[conflict_id], 'best redshift'] = 
vi.loc[vi.TargetID==unique_targets[conflict_id], 'VI class'] = 1.666667
#vi.loc[vi.TargetID==unique_targets[conflict_id], 'best spectype'] = 

#add a comment
vi.loc[vi.TargetID==unique_targets[conflict_id], 'merger comment'] = 'disagreement in flag, set to mean. poor sky subtraction.'

#look at the values again to make sure all is well
display(vi[vi.TargetID==unique_targets[conflict_id]])
print('conflict ID = ', conflict_id)


,TargetID,ExpID,Spec version,Redrock version,Redrock spectype,Redrock z,VI scanner,VI class,VI issue,VI z,...,DELTACHI2,best redshift,best spectype,vi_combined_flag,vi_diff,dz,vi_combined_z,all VI comments,N_VI,merger comment
490,35185796331865722,-1,0.33.0.dev3890,0,GALAXY,1.4936,SAE,1.666667,--,--,...,41.711448,1.4936,GALAXY,1.666667,2,0.0,1.4936,--|--|--,3,"disagreement in flag, set to mean. poor sky su..."
540,35185796331865722,-1,0.33.0.dev3890,0,GALAXY,1.4936,CGS,1.666667,--,--,...,41.711448,1.4936,GALAXY,1.666667,2,0.0,1.4936,--|--|--,3,"disagreement in flag, set to mean. poor sky su..."
590,35185796331865722,-1,0.33.0.dev3890,0,GALAXY,1.4936,ACE,1.666667,--,--,...,41.711448,1.4936,GALAXY,1.666667,2,0.0,1.4936,--|--|--,3,"disagreement in flag, set to mean. poor sky su..."


conflict ID =  40


In [103]:
#copy this text to a new cell to display the conflict
conflict_id=41
display_conflict(conflict_id, show_spectra=False)


,TargetID,Redrock spectype,Redrock z,VI scanner,VI class,VI issue,VI z,VI spectype,VI comment,best redshift,best spectype,vi_combined_flag,vi_diff,vi_combined_z,dz,all VI comments,N_VI,merger comment
496,35185796331866416,GALAXY,1.3877,SAE,1.0,--,--,--,calibration issue at 5800AA,1.3877,GALAXY,3.0,3,1.3877,0.0,calibration issue at 5800AA|--|--,3,none
546,35185796331866416,GALAXY,1.3877,CGS,4.0,--,--,--,--,1.3877,GALAXY,3.0,3,1.3877,0.0,calibration issue at 5800AA|--|--,3,none
596,35185796331866416,GALAXY,1.3877,ACE,4.0,--,--,--,--,1.3877,GALAXY,3.0,3,1.3877,0.0,calibration issue at 5800AA|--|--,3,none


In [104]:
#copy this text to a new cell to resolve the conflict
#vi.loc[vi.TargetID==unique_targets[conflict_id], 'best redshift'] = 
vi.loc[vi.TargetID==unique_targets[conflict_id], 'VI class'] = 3
#vi.loc[vi.TargetID==unique_targets[conflict_id], 'best spectype'] = 

#add a comment
vi.loc[vi.TargetID==unique_targets[conflict_id], 'merger comment'] = 'flag disagreement, take mean. poor sky subtraction'

#look at the values again to make sure all is well
display(vi[vi.TargetID==unique_targets[conflict_id]])
print('conflict ID = ', conflict_id)


,TargetID,ExpID,Spec version,Redrock version,Redrock spectype,Redrock z,VI scanner,VI class,VI issue,VI z,...,DELTACHI2,best redshift,best spectype,vi_combined_flag,vi_diff,dz,vi_combined_z,all VI comments,N_VI,merger comment
496,35185796331866416,-1,0.33.0.dev3890,0,GALAXY,1.3877,SAE,3.0,--,--,...,116.261356,1.3877,GALAXY,3.0,3,0.0,1.3877,calibration issue at 5800AA|--|--,3,"flag disagreement, take mean. poor sky subtrac..."
546,35185796331866416,-1,0.33.0.dev3890,0,GALAXY,1.3877,CGS,3.0,--,--,...,116.261356,1.3877,GALAXY,3.0,3,0.0,1.3877,calibration issue at 5800AA|--|--,3,"flag disagreement, take mean. poor sky subtrac..."
596,35185796331866416,-1,0.33.0.dev3890,0,GALAXY,1.3877,ACE,3.0,--,--,...,116.261356,1.3877,GALAXY,3.0,3,0.0,1.3877,calibration issue at 5800AA|--|--,3,"flag disagreement, take mean. poor sky subtrac..."


conflict ID =  41


In [105]:
#copy this text to a new cell to display the conflict
conflict_id=42
display_conflict(conflict_id, show_spectra=False)


,TargetID,Redrock spectype,Redrock z,VI scanner,VI class,VI issue,VI z,VI spectype,VI comment,best redshift,best spectype,vi_combined_flag,vi_diff,vi_combined_z,dz,all VI comments,N_VI,merger comment
598,35185796331867149,QSO,1.7975,SAE,3.0,--,1.7975,QSO,--,1.7975,QSO,3.0,2,1.7975,0.0,--|--|--|--,4,none
648,35185796331867149,QSO,1.7975,ACE,3.0,--,--,--,--,1.7975,QSO,3.0,2,1.7975,0.0,--|--|--|--,4,none
698,35185796331867149,QSO,1.7975,CGS,2.0,--,--,--,--,1.7975,QSO,3.0,2,1.7975,0.0,--|--|--|--,4,none
748,35185796331867149,QSO,1.7975,TWL,4.0,--,--,--,--,1.7975,QSO,3.0,2,1.7975,0.0,--|--|--|--,4,none


In [106]:
#copy this text to a new cell to resolve the conflict
#vi.loc[vi.TargetID==unique_targets[conflict_id], 'best redshift'] = 
vi.loc[vi.TargetID==unique_targets[conflict_id], 'VI class'] = 3
#vi.loc[vi.TargetID==unique_targets[conflict_id], 'best spectype'] = 

#add a comment
vi.loc[vi.TargetID==unique_targets[conflict_id], 'merger comment'] = 'flag disagreement, take mean.'

#look at the values again to make sure all is well
display(vi[vi.TargetID==unique_targets[conflict_id]])
print('conflict ID = ', conflict_id)


,TargetID,ExpID,Spec version,Redrock version,Redrock spectype,Redrock z,VI scanner,VI class,VI issue,VI z,...,DELTACHI2,best redshift,best spectype,vi_combined_flag,vi_diff,dz,vi_combined_z,all VI comments,N_VI,merger comment
598,35185796331867149,-1,0.33.0.dev3890,0,QSO,1.7975,SAE,3.0,--,1.7975,...,91.590343,1.7975,QSO,3.0,2,0.0,1.7975,--|--|--|--,4,"flag disagreement, take mean."
648,35185796331867149,-1,0.33.0.dev3890,0,QSO,1.7975,ACE,3.0,--,--,...,91.590343,1.7975,QSO,3.0,2,0.0,1.7975,--|--|--|--,4,"flag disagreement, take mean."
698,35185796331867149,-1,0.33.0.dev3890,0,QSO,1.7975,CGS,3.0,--,--,...,91.590343,1.7975,QSO,3.0,2,0.0,1.7975,--|--|--|--,4,"flag disagreement, take mean."
748,35185796331867149,-1,0.33.0.dev3890,0,QSO,1.7975,TWL,3.0,--,--,...,91.590343,1.7975,QSO,3.0,2,0.0,1.7975,--|--|--|--,4,"flag disagreement, take mean."


conflict ID =  42


In [107]:
#copy this text to a new cell to display the conflict
conflict_id=43
display_conflict(conflict_id, show_spectra=False)


,TargetID,Redrock spectype,Redrock z,VI scanner,VI class,VI issue,VI z,VI spectype,VI comment,best redshift,best spectype,vi_combined_flag,vi_diff,vi_combined_z,dz,all VI comments,N_VI,merger comment
601,35185796331868052,GALAXY,1.5414,SAE,1.0,--,--,--,Calib issue at 5800AA,1.5414,GALAXY,1.25,3,1.5414,0.0,Calib issue at 5800AA|No signal|--| [OII] on ...,4,none
651,35185796331868052,GALAXY,1.5414,ACE,0.0,--,--,--,No signal,1.5414,GALAXY,1.25,3,1.5414,0.0,Calib issue at 5800AA|No signal|--| [OII] on ...,4,none
701,35185796331868052,GALAXY,1.5414,CGS,1.0,--,--,--,--,1.5414,GALAXY,1.25,3,1.5414,0.0,Calib issue at 5800AA|No signal|--| [OII] on ...,4,none
751,35185796331868052,GALAXY,1.5414,TWL,3.0,--,--,--,[OII] on sky lines,1.5414,GALAXY,1.25,3,1.5414,0.0,Calib issue at 5800AA|No signal|--| [OII] on ...,4,none


In [108]:
#copy this text to a new cell to resolve the conflict
#vi.loc[vi.TargetID==unique_targets[conflict_id], 'best redshift'] = 
vi.loc[vi.TargetID==unique_targets[conflict_id], 'VI class'] = 1.25
#vi.loc[vi.TargetID==unique_targets[conflict_id], 'best spectype'] = 

#add a comment
vi.loc[vi.TargetID==unique_targets[conflict_id], 'merger comment'] = 'flag disagreement, take mean. poor sky subtraction'

#look at the values again to make sure all is well
display(vi[vi.TargetID==unique_targets[conflict_id]])
print('conflict ID = ', conflict_id)


,TargetID,ExpID,Spec version,Redrock version,Redrock spectype,Redrock z,VI scanner,VI class,VI issue,VI z,...,DELTACHI2,best redshift,best spectype,vi_combined_flag,vi_diff,dz,vi_combined_z,all VI comments,N_VI,merger comment
601,35185796331868052,-1,0.33.0.dev3890,0,GALAXY,1.5414,SAE,1.25,--,--,...,15.685232,1.5414,GALAXY,1.25,3,0.0,1.5414,Calib issue at 5800AA|No signal|--| [OII] on ...,4,"flag disagreement, take mean. poor sky subtrac..."
651,35185796331868052,-1,0.33.0.dev3890,0,GALAXY,1.5414,ACE,1.25,--,--,...,15.685232,1.5414,GALAXY,1.25,3,0.0,1.5414,Calib issue at 5800AA|No signal|--| [OII] on ...,4,"flag disagreement, take mean. poor sky subtrac..."
701,35185796331868052,-1,0.33.0.dev3890,0,GALAXY,1.5414,CGS,1.25,--,--,...,15.685232,1.5414,GALAXY,1.25,3,0.0,1.5414,Calib issue at 5800AA|No signal|--| [OII] on ...,4,"flag disagreement, take mean. poor sky subtrac..."
751,35185796331868052,-1,0.33.0.dev3890,0,GALAXY,1.5414,TWL,1.25,--,--,...,15.685232,1.5414,GALAXY,1.25,3,0.0,1.5414,Calib issue at 5800AA|No signal|--| [OII] on ...,4,"flag disagreement, take mean. poor sky subtrac..."


conflict ID =  43


In [109]:
#copy this text to a new cell to display the conflict
conflict_id=44
display_conflict(conflict_id, show_spectra=False)

,TargetID,Redrock spectype,Redrock z,VI scanner,VI class,VI issue,VI z,VI spectype,VI comment,best redshift,best spectype,vi_combined_flag,vi_diff,vi_combined_z,dz,all VI comments,N_VI,merger comment
605,35185796331868872,GALAXY,1.045,SAE,1.0,--,--,--,Calib issue at 5800AA,1.045,GALAXY,3.0,3,1.045,0.0,Calib issue at 5800AA|--|--|[OII] on sky lines,4,none
655,35185796331868872,GALAXY,1.045,ACE,4.0,--,--,--,--,1.045,GALAXY,3.0,3,1.045,0.0,Calib issue at 5800AA|--|--|[OII] on sky lines,4,none
705,35185796331868872,GALAXY,1.045,CGS,4.0,--,--,--,--,1.045,GALAXY,3.0,3,1.045,0.0,Calib issue at 5800AA|--|--|[OII] on sky lines,4,none
755,35185796331868872,GALAXY,1.045,TWL,3.0,--,--,--,[OII] on sky lines,1.045,GALAXY,3.0,3,1.045,0.0,Calib issue at 5800AA|--|--|[OII] on sky lines,4,none


In [110]:
#copy this text to a new cell to resolve the conflict
#vi.loc[vi.TargetID==unique_targets[conflict_id], 'best redshift'] = 
vi.loc[vi.TargetID==unique_targets[conflict_id], 'VI class'] = 3
#vi.loc[vi.TargetID==unique_targets[conflict_id], 'best spectype'] = 

#add a comment
vi.loc[vi.TargetID==unique_targets[conflict_id], 'merger comment'] = 'flag disagreement, take mean.'

#look at the values again to make sure all is well
display(vi[vi.TargetID==unique_targets[conflict_id]])
print('conflict ID = ', conflict_id)


,TargetID,ExpID,Spec version,Redrock version,Redrock spectype,Redrock z,VI scanner,VI class,VI issue,VI z,...,DELTACHI2,best redshift,best spectype,vi_combined_flag,vi_diff,dz,vi_combined_z,all VI comments,N_VI,merger comment
605,35185796331868872,-1,0.33.0.dev3890,0,GALAXY,1.045,SAE,3.0,--,--,...,8.820278,1.045,GALAXY,3.0,3,0.0,1.045,Calib issue at 5800AA|--|--|[OII] on sky lines,4,"flag disagreement, take mean."
655,35185796331868872,-1,0.33.0.dev3890,0,GALAXY,1.045,ACE,3.0,--,--,...,8.820278,1.045,GALAXY,3.0,3,0.0,1.045,Calib issue at 5800AA|--|--|[OII] on sky lines,4,"flag disagreement, take mean."
705,35185796331868872,-1,0.33.0.dev3890,0,GALAXY,1.045,CGS,3.0,--,--,...,8.820278,1.045,GALAXY,3.0,3,0.0,1.045,Calib issue at 5800AA|--|--|[OII] on sky lines,4,"flag disagreement, take mean."
755,35185796331868872,-1,0.33.0.dev3890,0,GALAXY,1.045,TWL,3.0,--,--,...,8.820278,1.045,GALAXY,3.0,3,0.0,1.045,Calib issue at 5800AA|--|--|[OII] on sky lines,4,"flag disagreement, take mean."


conflict ID =  44


In [111]:
#copy this text to a new cell to display the conflict
conflict_id=45
display_conflict(conflict_id, show_spectra=False)

,TargetID,Redrock spectype,Redrock z,VI scanner,VI class,VI issue,VI z,VI spectype,VI comment,best redshift,best spectype,vi_combined_flag,vi_diff,vi_combined_z,dz,all VI comments,N_VI,merger comment
623,35185796336062019,GALAXY,1.4408,SAE,0.0,--,--,--,--,1.4408,GALAXY,2.5,4,1.4408,0.0,--|--|--|--,4,none
673,35185796336062019,GALAXY,1.4408,ACE,3.0,--,--,--,--,1.4408,GALAXY,2.5,4,1.4408,0.0,--|--|--|--,4,none
723,35185796336062019,GALAXY,1.4408,CGS,3.0,--,--,--,--,1.4408,GALAXY,2.5,4,1.4408,0.0,--|--|--|--,4,none
773,35185796336062019,GALAXY,1.4408,TWL,4.0,--,--,--,--,1.4408,GALAXY,2.5,4,1.4408,0.0,--|--|--|--,4,none


In [112]:
#copy this text to a new cell to resolve the conflict
#vi.loc[vi.TargetID==unique_targets[conflict_id], 'best redshift'] = 
vi.loc[vi.TargetID==unique_targets[conflict_id], 'VI class'] = 4
#vi.loc[vi.TargetID==unique_targets[conflict_id], 'best spectype'] = 

#add a comment
vi.loc[vi.TargetID==unique_targets[conflict_id], 'merger comment'] = 'flag disagreement, but RR z is secure.'

#look at the values again to make sure all is well
display(vi[vi.TargetID==unique_targets[conflict_id]])
print('conflict ID = ', conflict_id)

,TargetID,ExpID,Spec version,Redrock version,Redrock spectype,Redrock z,VI scanner,VI class,VI issue,VI z,...,DELTACHI2,best redshift,best spectype,vi_combined_flag,vi_diff,dz,vi_combined_z,all VI comments,N_VI,merger comment
623,35185796336062019,-1,0.33.0.dev3890,0,GALAXY,1.4408,SAE,4.0,--,--,...,56.378119,1.4408,GALAXY,2.5,4,0.0,1.4408,--|--|--|--,4,"flag disagreement, but RR z is secure."
673,35185796336062019,-1,0.33.0.dev3890,0,GALAXY,1.4408,ACE,4.0,--,--,...,56.378119,1.4408,GALAXY,2.5,4,0.0,1.4408,--|--|--|--,4,"flag disagreement, but RR z is secure."
723,35185796336062019,-1,0.33.0.dev3890,0,GALAXY,1.4408,CGS,4.0,--,--,...,56.378119,1.4408,GALAXY,2.5,4,0.0,1.4408,--|--|--|--,4,"flag disagreement, but RR z is secure."
773,35185796336062019,-1,0.33.0.dev3890,0,GALAXY,1.4408,TWL,4.0,--,--,...,56.378119,1.4408,GALAXY,2.5,4,0.0,1.4408,--|--|--|--,4,"flag disagreement, but RR z is secure."


conflict ID =  45


In [113]:
#copy this text to a new cell to display the conflict
conflict_id=46
display_conflict(conflict_id, show_spectra=False)


,TargetID,Redrock spectype,Redrock z,VI scanner,VI class,VI issue,VI z,VI spectype,VI comment,best redshift,best spectype,vi_combined_flag,vi_diff,vi_combined_z,dz,all VI comments,N_VI,merger comment
631,35185796336062964,QSO,1.0142,SAE,3.0,--,1.0142,GALAXY,--,1.0142,GALAXY,3.75,1,1.0142,0.0,--|--|--|--,4,none
681,35185796336062964,QSO,1.0142,ACE,4.0,--,--,--,--,1.0142,QSO,3.75,1,1.0142,0.0,--|--|--|--,4,none
731,35185796336062964,QSO,1.0142,CGS,4.0,S,--,--,--,1.0142,QSO,3.75,1,1.0142,0.0,--|--|--|--,4,none
781,35185796336062964,QSO,1.0142,TWL,4.0,--,--,--,--,1.0142,QSO,3.75,1,1.0142,0.0,--|--|--|--,4,none


In [114]:
#copy this text to a new cell to resolve the conflict
#vi.loc[vi.TargetID==unique_targets[conflict_id], 'best redshift'] = 
#vi.loc[vi.TargetID==unique_targets[conflict_id], 'VI class'] = 
vi.loc[vi.TargetID==unique_targets[conflict_id], 'best spectype'] = 'QSO'

#add a comment
vi.loc[vi.TargetID==unique_targets[conflict_id], 'merger comment'] = 'disagreement in spectype. some broadened lines, eg Hb, set to QSO'

#look at the values again to make sure all is well
display(vi[vi.TargetID==unique_targets[conflict_id]])
print('conflict ID = ', conflict_id)

,TargetID,ExpID,Spec version,Redrock version,Redrock spectype,Redrock z,VI scanner,VI class,VI issue,VI z,...,DELTACHI2,best redshift,best spectype,vi_combined_flag,vi_diff,dz,vi_combined_z,all VI comments,N_VI,merger comment
631,35185796336062964,-1,0.33.0.dev3890,0,QSO,1.0142,SAE,3.0,--,1.0142,...,244.968604,1.0142,QSO,3.75,1,0.0,1.0142,--|--|--|--,4,disagreement in spectype. some broadened lines...
681,35185796336062964,-1,0.33.0.dev3890,0,QSO,1.0142,ACE,4.0,--,--,...,244.968604,1.0142,QSO,3.75,1,0.0,1.0142,--|--|--|--,4,disagreement in spectype. some broadened lines...
731,35185796336062964,-1,0.33.0.dev3890,0,QSO,1.0142,CGS,4.0,S,--,...,244.968604,1.0142,QSO,3.75,1,0.0,1.0142,--|--|--|--,4,disagreement in spectype. some broadened lines...
781,35185796336062964,-1,0.33.0.dev3890,0,QSO,1.0142,TWL,4.0,--,--,...,244.968604,1.0142,QSO,3.75,1,0.0,1.0142,--|--|--|--,4,disagreement in spectype. some broadened lines...


conflict ID =  46


In [115]:
#copy this text to a new cell to display the conflict
conflict_id=47
display_conflict(conflict_id, show_spectra=False)


,TargetID,Redrock spectype,Redrock z,VI scanner,VI class,VI issue,VI z,VI spectype,VI comment,best redshift,best spectype,vi_combined_flag,vi_diff,vi_combined_z,dz,all VI comments,N_VI,merger comment
644,35185796340253548,GALAXY,0.2797,SAE,3.0,S,0.2797,GALAXY,calibration issue; oversubtraction at 5800-68...,0.2797,GALAXY,3.5,1,0.2797,0.760569,calibration issue; oversubtraction at 5800-68...,4,none
694,35185796340253548,GALAXY,0.2797,ACE,4.0,R,1.2529,GALAXY,Poor continuum subtraction from 5800 to 6800A,1.2529,GALAXY,3.5,1,0.2797,0.760569,calibration issue; oversubtraction at 5800-68...,4,none
744,35185796340253548,GALAXY,0.2797,CGS,3.0,RS,1.2530,--,--,1.2530,GALAXY,3.5,1,0.2797,0.760569,calibration issue; oversubtraction at 5800-68...,4,none
794,35185796340253548,GALAXY,0.2797,TWL,4.0,RS,1.2530,--,Calibration features drive the best-fit,1.2530,GALAXY,3.5,1,0.2797,0.760569,calibration issue; oversubtraction at 5800-68...,4,none


In [116]:
#copy this text to a new cell to resolve the conflict
#vi.loc[vi.TargetID==unique_targets[conflict_id], 'best redshift'] = 
vi.loc[vi.TargetID==unique_targets[conflict_id], 'VI class'] = 3.5
#vi.loc[vi.TargetID==unique_targets[conflict_id], 'best spectype'] = 

#add a comment
vi.loc[vi.TargetID==unique_targets[conflict_id], 'merger comment'] = 'disagreement in z. 3/4 inspectors give same VI z. OII secure, take with flag 4. bad calibration.'

#look at the values again to make sure all is well
display(vi[vi.TargetID==unique_targets[conflict_id]])
print('conflict ID = ', conflict_id)

,TargetID,ExpID,Spec version,Redrock version,Redrock spectype,Redrock z,VI scanner,VI class,VI issue,VI z,...,DELTACHI2,best redshift,best spectype,vi_combined_flag,vi_diff,dz,vi_combined_z,all VI comments,N_VI,merger comment
644,35185796340253548,-1,0.33.0.dev3890,0,GALAXY,0.2797,SAE,3.5,S,0.2797,...,48.813303,0.2797,GALAXY,3.5,1,0.760569,0.2797,calibration issue; oversubtraction at 5800-68...,4,disagreement in z. 3/4 inspectors give same VI...
694,35185796340253548,-1,0.33.0.dev3890,0,GALAXY,0.2797,ACE,3.5,R,1.2529,...,48.813303,1.2529,GALAXY,3.5,1,0.760569,0.2797,calibration issue; oversubtraction at 5800-68...,4,disagreement in z. 3/4 inspectors give same VI...
744,35185796340253548,-1,0.33.0.dev3890,0,GALAXY,0.2797,CGS,3.5,RS,1.2530,...,48.813303,1.2530,GALAXY,3.5,1,0.760569,0.2797,calibration issue; oversubtraction at 5800-68...,4,disagreement in z. 3/4 inspectors give same VI...
794,35185796340253548,-1,0.33.0.dev3890,0,GALAXY,0.2797,TWL,3.5,RS,1.2530,...,48.813303,1.2530,GALAXY,3.5,1,0.760569,0.2797,calibration issue; oversubtraction at 5800-68...,4,disagreement in z. 3/4 inspectors give same VI...


conflict ID =  47


### and so on...

We should now recompute the conflicts, and not find any.

In [117]:
#vi_conflict = vi_gp.filter(lambda x: ( ( (x['VI class'].max()-x['VI class'].min()) >= 2) 
#                       | ( (x['best redshift'].max() - x['best redshift'].min()) / (1+x['best redshift'].min()) > 0.0033 ) 
#                       | (not all(i == x['best spectype'].iloc[0] for i in x['best spectype'])) )
#                       & (len(x) >= 2)) #x is a group by TargetID
vi_conflict = VI.find_conflicts(vi_gp)

In [118]:
unique_targets = np.unique(vi_conflict['TargetID'].tolist())
print('Targets with problematic VI: ', unique_targets)
print('Total number of conflicts to resolve: ', len(unique_targets))

Targets with problematic VI:  [35185796340253548]
Total number of conflicts to resolve:  1


Display anything that was missed (if "Total number of conflicts" isn't zero) and resolve!

In [119]:
#copy this text to a new cell to display the conflict
conflict_id=0
display_conflict(conflict_id)

,TargetID,Redrock spectype,Redrock z,VI scanner,VI class,VI issue,VI z,VI spectype,VI comment,best redshift,best spectype,vi_combined_flag,vi_diff,vi_combined_z,dz,all VI comments,N_VI,merger comment
644,35185796340253548,GALAXY,0.2797,SAE,3.5,S,0.2797,GALAXY,calibration issue; oversubtraction at 5800-68...,0.2797,GALAXY,3.5,1,0.2797,0.760569,calibration issue; oversubtraction at 5800-68...,4,disagreement in z. 3/4 inspectors give same VI...
694,35185796340253548,GALAXY,0.2797,ACE,3.5,R,1.2529,GALAXY,Poor continuum subtraction from 5800 to 6800A,1.2529,GALAXY,3.5,1,0.2797,0.760569,calibration issue; oversubtraction at 5800-68...,4,disagreement in z. 3/4 inspectors give same VI...
744,35185796340253548,GALAXY,0.2797,CGS,3.5,RS,1.2530,--,--,1.2530,GALAXY,3.5,1,0.2797,0.760569,calibration issue; oversubtraction at 5800-68...,4,disagreement in z. 3/4 inspectors give same VI...
794,35185796340253548,GALAXY,0.2797,TWL,3.5,RS,1.2530,--,Calibration features drive the best-fit,1.2530,GALAXY,3.5,1,0.2797,0.760569,calibration issue; oversubtraction at 5800-68...,4,disagreement in z. 3/4 inspectors give same VI...


In [120]:
print(vi.loc[644 ]['merger comment'])


disagreement in z. 3/4 inspectors give same VI z. OII secure, take with flag 4. bad calibration.


In [121]:
#copy this text to a new cell to resolve the conflict
vi.loc[vi.TargetID==unique_targets[conflict_id], 'best redshift'] = 1.2530
vi.loc[vi.TargetID==unique_targets[conflict_id], 'VI class'] = 4
#vi.loc[vi.TargetID==unique_targets[conflict_id], 'best spectype'] = 

#add a comment
#vi.loc[vi.TargetID==unique_targets[conflict_id], 'merger comment'] = 

#look at the values again to make sure all is well
display(vi[vi.TargetID==unique_targets[conflict_id]])
print('conflict ID = ', conflict_id)

,TargetID,ExpID,Spec version,Redrock version,Redrock spectype,Redrock z,VI scanner,VI class,VI issue,VI z,...,DELTACHI2,best redshift,best spectype,vi_combined_flag,vi_diff,dz,vi_combined_z,all VI comments,N_VI,merger comment
644,35185796340253548,-1,0.33.0.dev3890,0,GALAXY,0.2797,SAE,4.0,S,0.2797,...,48.813303,1.253,GALAXY,3.5,1,0.760569,0.2797,calibration issue; oversubtraction at 5800-68...,4,disagreement in z. 3/4 inspectors give same VI...
694,35185796340253548,-1,0.33.0.dev3890,0,GALAXY,0.2797,ACE,4.0,R,1.2529,...,48.813303,1.253,GALAXY,3.5,1,0.760569,0.2797,calibration issue; oversubtraction at 5800-68...,4,disagreement in z. 3/4 inspectors give same VI...
744,35185796340253548,-1,0.33.0.dev3890,0,GALAXY,0.2797,CGS,4.0,RS,1.2530,...,48.813303,1.253,GALAXY,3.5,1,0.760569,0.2797,calibration issue; oversubtraction at 5800-68...,4,disagreement in z. 3/4 inspectors give same VI...
794,35185796340253548,-1,0.33.0.dev3890,0,GALAXY,0.2797,TWL,4.0,RS,1.2530,...,48.813303,1.253,GALAXY,3.5,1,0.760569,0.2797,calibration issue; oversubtraction at 5800-68...,4,disagreement in z. 3/4 inspectors give same VI...


conflict ID =  0


Let's check the number of conflicts again.

In [122]:
#vi_conflict = vi_gp.filter(lambda x: ( ( (x['VI class'].max()-x['VI class'].min()) >= 2) 
#                       | ( (x['best redshift'].max() - x['best redshift'].min()) / (1+x['best redshift'].min()) > 0.0033 ) 
#                       | (not all(i == x['best spectype'].iloc[0] for i in x['best spectype'])) )
#                       & (len(x) >= 2)) #x is a group by TargetID
vi_conflict = VI.find_conflicts(vi_gp)

In [123]:
unique_targets = np.unique(vi_conflict['TargetID'].tolist())
print('Targets with problematic VI: ', unique_targets)
print('Total number of conflicts to resolve: ', len(unique_targets))

Targets with problematic VI:  []
Total number of conflicts to resolve:  0


## Woohoo!

## Finally
we need to recompute vi_combined_flag and vi_combined_z, after everything is resolved.

In [125]:
vi['vi_combined_flag'] = vi.groupby('TargetID')['VI class'].transform('mean')
vi['vi_combined_z'] = vi.groupby('TargetID')['best redshift'].transform('mean')


## Now we prepare to write to file. 

### The important columns for the truth table construction are **vi_combined_flag** and **vi_combined_z** 

The truth table should **take the redhift value in vi_combined_z** for all objects with **vi_combined_flag >= 2.5** (for discussion, but this catches the cases where there is some disagreement on the 2/3 boundary).

After the merging process, those two columns will be the same for ALL of the members of each group by TargetID, so it doesn't matter which member we write to file. Taking the 1st one just makes this easy. 


In [126]:
vi.keys()

Index(['TargetID', 'ExpID', 'Spec version', 'Redrock version',
       'Redrock spectype', 'Redrock z', 'VI scanner', 'VI class', 'VI issue',
       'VI z', 'VI spectype', 'VI comment', 'FIBER', 'FLUX_G', 'FLUX_R',
       'FLUX_Z', 'FIBERFLUX_G', 'FIBERFLUX_R', 'FIBERFLUX_Z', 'DELTACHI2',
       'best redshift', 'best spectype', 'vi_combined_flag', 'vi_diff', 'dz',
       'vi_combined_z', 'all VI comments', 'N_VI', 'merger comment'],
      dtype='object')

In [127]:
print(VI_dir+save_filename)

/global/homes/r/rtojeiro/SV/VI_files/SV0/ELG/truth_table_ELGs_sets46_50.txt


In [128]:
#vi_gp['Redrock z', 'Redrock spectype', 'best redshift' , 'best spectype', 'vi_combined_flag', 'all VI comments', 'merger comment', 'N_VI', 'DELTACHI2','FIBER','FLUX_G','FLUX_R','FLUX_Z','FIBERFLUX_G','FIBERFLUX_R','FIBERFLUX_Z'].first().to_csv(VI_dir+'merged_VI.txt')
vi_gp['Redrock z', 'Redrock spectype', 'best redshift' , 'best spectype', 'vi_combined_flag', 'N_VI', 'all VI comments', 'merger comment','DELTACHI2','FIBER','FLUX_G','FLUX_R','FLUX_Z','FIBERFLUX_G','FIBERFLUX_R','FIBERFLUX_Z'].first().to_csv(VI_dir+save_filename)

Check that merged file reads in OK - check comments

In [129]:
merged_file = pd.read_csv(VI_dir+save_filename)

In [130]:
merged_file.keys()

Index(['TargetID', 'Redrock z', 'Redrock spectype', 'best redshift',
       'best spectype', 'vi_combined_flag', 'N_VI', 'all VI comments',
       'merger comment', 'DELTACHI2', 'FIBER', 'FLUX_G', 'FLUX_R', 'FLUX_Z',
       'FIBERFLUX_G', 'FIBERFLUX_R', 'FIBERFLUX_Z'],
      dtype='object')

In [131]:
merged_file

,TargetID,Redrock z,Redrock spectype,best redshift,best spectype,vi_combined_flag,N_VI,all VI comments,merger comment,DELTACHI2,FIBER,FLUX_G,FLUX_R,FLUX_Z,FIBERFLUX_G,FIBERFLUX_R,FIBERFLUX_Z
0,35185790300456650,0.9039,GALAXY,0.9039,GALAXY,4.000000,3,--| MgII in absorption|--,none,687.608811,3619,0.663110,1.218882,3.526391,0.340052,0.625059,1.808382
1,35185790300456731,1.0406,GALAXY,1.0406,GALAXY,4.000000,3,--|--|--,none,228.047977,3630,0.739689,1.150407,3.192757,0.264269,0.411006,1.140678
2,35185790300456806,0.8467,GALAXY,0.8467,GALAXY,4.000000,3,--|--|--,none,1189.062399,3647,0.636498,1.023776,2.591709,0.324563,0.522043,1.321562
3,35185790300456962,1.7394,QSO,1.7394,QSO,1.000000,3,--| Based on very weak OII|--,none,5.529293,3635,0.443731,0.504769,1.035124,0.155869,0.177310,0.363607
4,35185790300457011,0.7396,GALAXY,1.3366,GALAXY,4.000000,3,--| Redrock confuses OII for OII.|--,disagreement in z. 2/3 inspectors give secure ...,8.907231,3631,0.347633,0.271157,0.389936,0.225924,0.176222,0.253416
5,35185790300457140,0.8185,GALAXY,0.8185,GALAXY,3.666667,3,--|--|--,none,463.025039,3601,0.409779,0.842773,2.316698,0.214424,0.440996,1.212255
6,35185790300457266,1.0670,GALAXY,1.0670,GALAXY,4.000000,3,--|--|--,none,472.154292,3604,0.822647,1.066252,2.078273,0.323192,0.418897,0.816489
7,35185790300457278,0.7326,GALAXY,0.7326,GALAXY,3.666667,3,--|--|--,none,445.100228,3648,0.326999,0.409055,0.675204,0.182182,0.227898,0.376178
8,35185790300457774,0.8583,GALAXY,0.8583,GALAXY,3.666667,3,--|--|--,none,157.756383,3606,0.679739,1.498361,4.811672,0.191092,0.421228,1.352685
9,35185790300457928,0.8820,GALAXY,0.8820,GALAXY,4.000000,3,--|--|--,none,1472.805541,3796,0.716332,0.986877,2.367269,0.374784,0.516333,1.238552


In [132]:
merged_file.loc[9]['merger comment']

'none'

In [133]:
merged_file.loc[9]['all VI comments']

'--|--|--'

In [134]:
VI_dir

'/global/homes/r/rtojeiro/SV/VI_files/SV0/ELG/'